In [1]:
# Copyright 2024 Daniel Franzen and Jan Disselhoff
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [2]:
# This notebook contains our winning submission to the ARC Prize 2025 Kaggle competition,
# scoring 53.5 points on the private evaluation set.
# the ARChitects (Daniel Franzen and Jan Disselhoff)

ARC2024 1등 code를 수정함

In [ ]:
%%writefile model_runner.py
import json
import os, sys
import bz2
import pickle
import numpy as np
from tqdm import tqdm

def indices_required_for_merges(keep_indices, vocab, merges):
    """
    BPE 병합에 필요한 모든 토큰 인덱스를 찾는 함수
    
    Args:
        keep_indices: 유지할 토큰 인덱스들의 딕셔너리
        vocab: 어휘 사전
        merges: BPE 병합 규칙들
    
    Returns:
        병합에 필요한 모든 인덱스가 포함된 딕셔너리
    """
    merges_lookup = {}
    # 각 병합 규칙에서 필요한 하위 토큰들을 매핑
    for m in merges:
        a, b = m.split(' ') if isinstance(m, str) else m
        key = vocab[f'{a}{b}']
        if key not in merges_lookup: merges_lookup[key] = set()
        merges_lookup[key].add(vocab[a])
        merges_lookup[key].add(vocab[b])
    
    # 재귀적으로 필요한 모든 토큰 인덱스 수집
    to_process = list(keep_indices)
    while len(to_process):
        for w in merges_lookup.get(to_process.pop(), []):
            if w not in keep_indices:
                keep_indices[w] = None
                to_process.append(w)
    return keep_indices

def remove_unused_merges(merges, vocab):
    """
    사용되지 않는 BPE 병합 규칙들을 제거하는 함수
    
    Args:
        merges: 병합 규칙 리스트
        vocab: 어휘 사전
    
    Returns:
        유효한 병합 규칙들만 포함된 리스트
    """
    return [f'{a} {b}' for a, b in [m.split(' ') if isinstance(m, str) else m for m in merges] 
            if all(w in vocab for w in [a, b, a + b])]

def map_special_tokens(data, mapping=None):
    """
    특별 토큰들을 매핑하거나 수집하는 함수
    
    Args:
        data: 토큰 데이터 (딕셔너리 또는 리스트)
        mapping: 토큰 인덱스 매핑 (선택사항)
    
    Returns:
        특별 토큰 인덱스들의 집합
    """
    tokens = set()
    if isinstance(data, dict):
        special = data.get('special_tokens')
        if special is not None:
            for v in special.values():
                tokens.update(v['ids'])
                # 매핑이 제공된 경우 토큰 ID들을 새로운 인덱스로 변환
                if mapping is not None:
                    v['ids'] = [mapping.get(i) for i in v['ids'] if i in mapping]
    
    # 재귀적으로 중첩된 데이터 구조 처리
    for v in (data.values() if isinstance(data, dict) else data if isinstance(data, list) else []):
        tokens.update(map_special_tokens(v, mapping))
    return tokens

def remove_tokenizer_normalizer(tokenizer):
    """
    토크나이저의 정규화 기능을 제거하는 함수
    
    Args:
        tokenizer: HuggingFace 토크나이저
    """
    from tokenizers import Tokenizer
    assert tokenizer.is_fast
    tokenizer_json = json.loads(tokenizer._tokenizer.to_str())
    if tokenizer_json.get('normalizer') is not None:
        tokenizer_json['normalizer'] = None
        tokenizer._tokenizer = Tokenizer.from_str(json.dumps(tokenizer_json))

def shrink_tokenizer_vocab(tokenizer, keep_indices, keep_special_tokens, keep_token_order):
    """
    토크나이저의 어휘를 축소하는 함수
    
    Args:
        tokenizer: 토크나이저 객체
        keep_indices: 유지할 토큰 인덱스들
        keep_special_tokens: 특별 토큰 유지 여부
        keep_token_order: 토큰 순서 유지 여부
    
    Returns:
        (매핑 딕셔너리, 유지된 인덱스들)
    """
    from tokenizers import Tokenizer
    assert tokenizer.is_fast
    tokenizer_json = json.loads(tokenizer._tokenizer.to_str())
    assert tokenizer_json['model']['type'] == "BPE"
    
    # 특별 토큰들 추가
    if keep_special_tokens:
        keep_indices.update({k: None for k in tokenizer.all_special_ids})
        keep_indices.update({k: None for k in map_special_tokens(tokenizer_json.get('post_processor'))})
    
    # BPE 병합에 필요한 모든 인덱스 포함
    keep_indices = indices_required_for_merges(keep_indices, tokenizer_json['model']['vocab'], tokenizer_json['model']['merges'])
    
    # 토큰 순서 정렬
    if keep_token_order: keep_indices = sorted(keep_indices)
    
    # 새로운 인덱스 매핑 생성
    mapping = {old: new for new, old in enumerate(keep_indices)}
    
    # 어휘 사전 업데이트
    tokenizer_json['model']['vocab'] = {k: mapping[v] for k, v in tokenizer_json['model']['vocab'].items() if v in mapping}
    tokenizer_json['model']['merges'] = remove_unused_merges(tokenizer_json['model']['merges'], tokenizer_json['model']['vocab'])
    
    # 추가된 토큰들 업데이트
    special_tokens_order = [t['id'] for t in tokenizer_json['added_tokens']]
    assert special_tokens_order==sorted(special_tokens_order)
    tokenizer_json['added_tokens'] = sorted([{**t, 'id': mapping[t['id']]} for t in tokenizer_json['added_tokens'] if t['id'] in mapping], key=lambda t: t['id'])
    
    # 후처리기 업데이트
    map_special_tokens(tokenizer_json.get('post_processor'), mapping)
    tokenizer._tokenizer = Tokenizer.from_str(json.dumps(tokenizer_json))
    return mapping, keep_indices

def shrink_model_embeddings(model, keep_indices, mapping):
    """
    모델의 임베딩 레이어를 축소하는 함수
    
    Args:
        model: 언어 모델
        keep_indices: 유지할 인덱스들
        mapping: 토큰 인덱스 매핑
    """
    import torch
    with torch.no_grad():
        # 유지할 토큰들만 선택
        row_select = torch.tensor(list(keep_indices))
        
        # 입력 임베딩 축소
        new_embed_t = torch.index_select(model.get_input_embeddings().weight.data, 0, row_select.to(model.get_input_embeddings().weight.data.device))
        # 출력 임베딩 축소
        new_lm_head = torch.index_select(model.get_output_embeddings().weight.data, 0, row_select.to(model.get_output_embeddings().weight.data.device))
        
        # 모델 크기 조정
        model.resize_token_embeddings(len(keep_indices))
        model.get_input_embeddings().weight.data[:] = new_embed_t
        model.get_output_embeddings().weight.data[:] = new_lm_head
        
        # 모델 설정의 특별 토큰 ID들 업데이트
        for config in [model.config, model.generation_config]:
            for k, v in list(config.to_dict().items()):
                if k.endswith('token_id'):
                    setattr(config, k, [mapping.get(t) for t in v] if isinstance(v, list) else mapping.get(v))

def shrink_embeddings(model, tokenizer, corpus=None, keep_token_ids=[], keep_tokens=[], remove_token_ids=[], keep_model_tokens=True, keep_special_tokens=True, keep_normalizer=False, keep_token_order=True):
    """
    모델과 토크나이저의 임베딩을 축소하는 메인 함수
    
    Args:
        model: 언어 모델
        tokenizer: 토크나이저
        corpus: 분석할 코퍼스 (선택사항)
        keep_token_ids: 유지할 토큰 ID 리스트
        keep_tokens: 유지할 토큰 문자열 리스트
        remove_token_ids: 제거할 토큰 ID 리스트
        keep_model_tokens: 모델 토큰 유지 여부
        keep_special_tokens: 특별 토큰 유지 여부
        keep_normalizer: 정규화기 유지 여부
        keep_token_order: 토큰 순서 유지 여부
    
    Returns:
        토큰 인덱스 매핑 딕셔너리
    """
    if not keep_normalizer: remove_tokenizer_normalizer(tokenizer)
    from collections import OrderedDict  # 순서가 있는 집합으로 사용
    keep_indices = OrderedDict()
    
    # 유지할 토큰들 수집
    keep_indices.update({k: None for k in keep_token_ids})
    keep_indices.update({tokenizer.vocab[t]: None for t in keep_tokens})
    if corpus is not None: keep_indices.update({k: None for k in tokenizer(corpus)['input_ids']})
    
    # 모델에서 사용되는 토큰들 유지
    if keep_model_tokens:
        for config in [model.config, model.generation_config]:
            for k, v in config.to_dict().items():
                if k.endswith('token_id'):
                    keep_indices.update({k: None for k in (v if isinstance(v, list) else [v])})
    
    # None 값과 제거할 토큰들 정리
    keep_indices.pop(None, None)
    for idx in remove_token_ids: keep_indices.pop(idx, None)
    
    # 토크나이저와 모델 축소 실행
    mapping, keep_indices = shrink_tokenizer_vocab(tokenizer, keep_indices, keep_special_tokens, keep_token_order)
    shrink_model_embeddings(model, keep_indices, mapping=mapping)
    return mapping

def fix_dtypes(model, fix_weights=True, fix_quant_states=True):
    """
    모델의 데이터 타입을 수정하는 함수
    
    Args:
        model: 언어 모델
        fix_weights: 가중치 타입 수정 여부
        fix_quant_states: 양자화 상태 타입 수정 여부
    
    Returns:
        수정된 모델
    """
    import torch
    for module in model.modules():
        weight = getattr(module, 'weight', None)
        if weight is not None:
            if torch.is_floating_point(weight):
                # 부동소수점 가중치 타입 수정
                if fix_weights and weight.dtype!=model.dtype:
                    module.to(model.dtype)
            else:
                # 양자화된 가중치의 상태 타입 수정
                qs = getattr(weight, 'quant_state', None)
                if qs is not None:
                    if fix_quant_states and qs.dtype!=model.dtype:
                        qs.dtype = model.dtype
    return model

def merge_peft_into_base(model):
    """
    PEFT(Parameter Efficient Fine-Tuning) 모델을 베이스 모델에 병합하는 함수
    
    Args:
        model: PEFT 모델
    
    Returns:
        병합된 베이스 모델
    """
    print('*** PEFT 모델을 베이스 모델에 병합 중...')
    assert is_peft_model(model)
    return fix_dtypes(model.merge_and_unload())

def save_model(store_path, model=None, tokenizer=None, merge=False):
    """
    모델과 토크나이저를 저장하는 함수
    
    Args:
        store_path: 저장 경로
        model: 저장할 모델 (선택사항)
        tokenizer: 저장할 토크나이저 (선택사항)
        merge: PEFT 모델 병합 여부
    
    Returns:
        처리된 모델
    """
    if merge: model = merge_peft_into_base(model)
    if store_path is not None:
        assert model is not None or tokenizer is not None
        print(f"*** {'병합된 ' if merge else ''}모델/토크나이저를 '{store_path}'에 저장 중...")
        if model is not None: model.save_pretrained(store_path)
        if tokenizer is not None:
            tokenizer.save_pretrained(store_path)
            # 불필요한 tokenizer.model 파일 삭제
            to_delete = os.path.join(store_path, 'tokenizer.model')
            if os.path.isfile(to_delete): os.remove(to_delete)
    return model

def is_unsloth_model(model):
    """Unsloth 모델인지 확인하는 함수"""
    return model.model_tags is not None and 'unsloth' in model.model_tags

def is_peft_model(model):
    """PEFT 모델인지 확인하는 함수"""
    return hasattr(model, 'peft_type')

def download_model(repo_id, store_path, get_name=lambda n: os.path.join(n.replace('/', '--'), 'transformers', 'default', '1')):
    """
    HuggingFace에서 모델을 다운로드하는 함수
    
    Args:
        repo_id: HuggingFace 모델 저장소 ID
        store_path: 로컬 저장 경로
        get_name: 파일명 생성 함수
    
    Returns:
        모델 경로
    """
    import os
    if os.path.exists(repo_id): return repo_id
    model_path = os.path.join(store_path, get_name(repo_id))
    if not os.path.exists(model_path):
        from huggingface_hub import snapshot_download
        download_path = snapshot_download(repo_id=repo_id)
        os.makedirs(os.path.split(model_path)[0], exist_ok=True)
        os.symlink(download_path, model_path, target_is_directory=True)
    return model_path

def get_and_fix_peft_weights(store):
    """
    PEFT 가중치를 로드하고 수정하는 함수
    
    Args:
        store: PEFT 가중치 저장 경로
    
    Returns:
        수정된 state_dict
    """
    print(f"*** '{store}'에서 PEFT state_dict 로드 중...")
    from peft import load_peft_weights
    state_dict = load_peft_weights(store)
    # 불필요한 modules_to_save 관련 키들 제거
    for k in list(state_dict.keys()):
        if 'modules_to_save' in k:
            del state_dict[k]
            original_module_key = k.replace('.modules_to_save.', '.original_module.')
            if original_module_key in state_dict: del state_dict[original_module_key]
            assert k.replace('.modules_to_save.', '.') in state_dict
    return state_dict

def set_peft_weights(model, state_dict):
    """
    모델에 PEFT 가중치를 설정하는 함수
    
    Args:
        model: 타겟 모델
        state_dict: PEFT 가중치 딕셔너리
    """
    print(f"*** 모델 state_dict 설정 중...")
    from peft import set_peft_model_state_dict
    res = set_peft_model_state_dict(model, state_dict)
    assert not res.unexpected_keys

def load_peft_state(model, store):
    """
    저장된 PEFT 상태를 모델에 로드하는 함수
    
    Args:
        model: 타겟 모델
        store: PEFT 상태 저장 경로
    """
    set_peft_weights(model, get_and_fix_peft_weights(store))

def prepare_model(model, mode, tokenizer=None, formatter=None, shrink_embedding=False, dequantize=False, peft=[], local_files_only=False, add_special_tokens={}, set_pad_token=None, keep_tokens=[], keep_normalizer=None, peft_trainable=True, device_map=None, tf_grad_cp=True, tf_use_fa2=True, **kwargs):
    """
    모델과 토크나이저를 준비하는 메인 함수
    
    Args:
        model: 모델 경로 또는 모델 객체
        mode: 로드 모드 ('unsloth_4bit', 'transformers', 'transformers_bf16', 등)
        tokenizer: 토크나이저 (선택사항)
        formatter: 데이터 포맷터 (선택사항)
        shrink_embedding: 임베딩 축소 여부
        dequantize: 역양자화 여부
        peft: PEFT 설정 리스트
        local_files_only: 로컬 파일만 사용 여부
        add_special_tokens: 추가할 특별 토큰들
        set_pad_token: 패딩 토큰 설정
        keep_tokens: 유지할 토큰들
        keep_normalizer: 정규화기 유지 여부
        peft_trainable: PEFT 훈련 가능 여부
        device_map: 디바이스 매핑
        tf_grad_cp: 그래디언트 체크포인팅 사용 여부
        tf_use_fa2: Flash Attention 2 사용 여부
    
    Returns:
        (모델, 토크나이저, 포맷터) 튜플
    """
    if isinstance(model, str):
        assert tokenizer is None
        print(f"*** '{model}'에서 베이스 모델과 토크나이저 로드 중...")
        
        if mode=='unsloth_4bit':
            assert device_map is None, '지원되지 않음'
            from unsloth import FastLanguageModel
            model, tokenizer = FastLanguageModel.from_pretrained(model_name=model, dtype=None, load_in_4bit=True, local_files_only=local_files_only, **kwargs)
        
        elif mode in ['transformers', 'transformers_bf16', 'transformers_4bit', 'transformers_bf16_4bit', 'tokenizer_only']:
            import torch
            model_load_args = {}
            if device_map is not None: model_load_args['device_map'] = device_map
            if tf_use_fa2: model_load_args['attn_implementation'] = 'flash_attention_2'
            if mode in ['transformers_bf16', 'transformers_bf16_4bit']: model_load_args['torch_dtype'] = torch.bfloat16
            elif mode in ['transformers_4bit', 'transformers_bf16_4bit']:
                from transformers import BitsAndBytesConfig
                nf4_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type='nf4', bnb_4bit_use_double_quant=True, bnb_4bit_compute_dtype=torch.bfloat16)
                model_load_args['quantization_config'] = nf4_config
            
            from transformers import AutoTokenizer, AutoModelForCausalLM
            tokenizer = AutoTokenizer.from_pretrained(model, local_files_only=local_files_only, **kwargs)
            model = AutoModelForCausalLM.from_pretrained(model, **model_load_args) if mode!='tokenizer_only' else None
            if tf_grad_cp and model is not None: model.gradient_checkpointing_enable()
        else: 
            raise NotImplementedError('알 수 없는 모드입니다.')
    
    # 특별 토큰 추가
    if add_special_tokens: tokenizer.add_special_tokens(add_special_tokens)
    if set_pad_token is not None: tokenizer.pad_token = set_pad_token
    
    # 포맷터 초기화
    if formatter is not None and not hasattr(formatter, 'corpus'):
        formatter = formatter(tokenizer=tokenizer)
    
    # 임베딩 축소
    if (shrink_embedding<len(tokenizer.vocab) if type(shrink_embedding)==int else shrink_embedding) or keep_normalizer is False:
        print('*** 임베딩 축소 중...')
        embedding_size_before_shrink = len(tokenizer.vocab)
        mapping = shrink_embeddings(model, tokenizer, formatter.get_corpus(), keep_tokens=keep_tokens, keep_normalizer=keep_normalizer)
        print(f'*** -> 임베딩 크기를 {embedding_size_before_shrink}에서 {len(mapping)} 단어로 축소했습니다.')
    
    # 역양자화
    if dequantize:
        print(f'*** 모델 역양자화 중...')
        model = model.dequantize()
    
    # PEFT 설정
    if len(peft):
        peft_trained = True if is_peft_model(model) else None
        for i, m in enumerate(peft):
            if peft_trained is True: model, peft_trained = merge_peft_into_base(model), None
            if isinstance(m, str):
                if peft_trained is False:
                    _, peft_trained = load_peft_state(model, m), True
                else:
                    print(f"*** '{m}'에서 PEFT 모델 로드 중...")
                    # unsloth 사용 시 주의: PeftModel로 로드하면 unsloth 최적화가 적용되지 않음
                    from peft import PeftModel
                    model, peft_trained = PeftModel.from_pretrained(model, m, trainable=peft_trainable), True
            else:
                assert peft_trained is None
                if isinstance(m, dict):
                    print('*** 새 PEFT 모델 생성 중...')
                    if is_unsloth_model(model):
                        from unsloth import FastLanguageModel
                        my_get_peft_model = FastLanguageModel.get_peft_model
                    else:
                        from peft import LoraConfig, get_peft_model
                        my_get_peft_model = lambda model, **kwargs: get_peft_model(model, LoraConfig(**kwargs))
                    model, peft_trained = my_get_peft_model(model, **m), False
                else: assert m is None
    
    return model, tokenizer, formatter

def training_run(model, formatter, dataset, train_args, max_seq_length, merge=False, store=None, packing=False, grad_acc_fix=False, optimizers=None):
    """
    모델 훈련을 실행하는 함수
    
    Args:
        model: 훈련할 모델
        formatter: 데이터 포맷터
        dataset: 훈련 데이터셋
        train_args: 훈련 인자들
        max_seq_length: 최대 시퀀스 길이
        merge: 훈련 후 병합 여부
        store: 모델 저장 경로
        packing: 시퀀스 패킹 사용 여부
        grad_acc_fix: 그래디언트 누적 수정 여부
        optimizers: 최적화 도구
    
    Returns:
        (모델, 훈련 통계) 튜플
    """
    assert merge is False, "훈련 후 병합은 작동하지 않는 것으로 보임 (적어도 unsloth에서는 저장된 병합 모델이 훈련되지 않은 가중치를 포함함!)"
    import torch
    from datasets import Dataset
    add_train_args = {}
    
    # Unsloth 또는 일반 Transformers 설정
    if is_unsloth_model(model):
        from unsloth import FastLanguageModel
        from unsloth import UnslothTrainer as Trainer
        from unsloth import UnslothTrainingArguments as TrainingArguments
        from unsloth import is_bfloat16_supported
        FastLanguageModel.for_training(model)
        add_train_args.update(fp16=not is_bfloat16_supported(), bf16=is_bfloat16_supported())
    else:
        from trl import SFTConfig as TrainingArguments
        from trl import SFTTrainer as Trainer
        model.train()
        add_train_args.update(bf16=True)

    # 토크나이저 설정
    formatter.tokenizer.padding_side = 'right'
    
    # Unsloth 모델의 임베딩을 float32로 변환
    if is_unsloth_model(model):
        for convert_to_float in [model.get_input_embeddings(), model.get_output_embeddings()]:
            if convert_to_float.weight.dtype!=torch.float32: convert_to_float.to(torch.float32)

    add_args = {}
    if optimizers is not None: add_args['optimizers'] = optimizers

    # 트레이너 설정
    trainer = Trainer(
        model=model,
        tokenizer=formatter.tokenizer,
        data_collator=formatter.get_data_collator(),
        train_dataset=Dataset.from_list(dataset.as_list(formatter)),
        dataset_text_field="text",
        max_seq_length=max_seq_length,
        dataset_num_proc=None,
        packing=packing,  # 짧은 시퀀스에 대해 훈련 속도를 5배 향상시킬 수 있음
        **add_args,
        args=TrainingArguments(
            **add_train_args,
            **train_args
        ),
    )

    print('*** 훈련 실행 시작...')
    # 그래디언트 누적 수정 사용 여부에 따른 훈련 실행
    if grad_acc_fix and is_unsloth_model(model):
        from unsloth import unsloth_train
        trainer_stats = unsloth_train(trainer)
    else:
        if is_unsloth_model(model) and train_args['gradient_accumulation_steps']>1: 
            print('*** 경고: 결함이 있는 unsloth 그래디언트 누적을 사용 중')
        trainer_stats = trainer.train()
    
    try: print(f'*** -> 훈련이 {trainer_stats.metrics["train_runtime"]}초 소요되었습니다.')
    except: pass
    
    if store is not None: save_model(store, model, formatter.tokenizer, merge=merge)
    return model, trainer_stats

def inference_load(store, keys=True, result_dict=None, always_read_from_file=False):
    """
    저장된 추론 결과를 로드하는 함수
    
    Args:
        store: 결과 저장 경로
        keys: 로드할 키들 (True면 모든 키)
        result_dict: 결과를 저장할 딕셔너리
        always_read_from_file: 항상 파일에서 읽을지 여부
    
    Returns:
        로드된 결과 딕셔너리
    """
    if result_dict is None: result_dict = {}
    if store is not None:
        if keys is True: keys = os.listdir(store)
        for key in keys:
            if always_read_from_file or key not in result_dict:
                try:
                    with bz2.BZ2File(os.path.join(store, key)) as f: 
                        result_dict[key] = pickle.load(f)
                except: continue
    return result_dict

def inference_save(store, key, outputs):
    """
    추론 결과를 저장하는 함수
    
    Args:
        store: 저장 경로
        key: 결과 키
        outputs: 저장할 출력 결과
    """
    if store is not None:
        os.makedirs(store, exist_ok=True)
        with bz2.BZ2File(os.path.join(store, key), 'w') as f: 
            pickle.dump(outputs, f)

class Decoder(object):
    """
    모델 추론 결과를 디코딩하고 평가하는 클래스
    """
    def __init__(self, formatter, dataset, n_guesses, max_outputs=None, frac_score=False, quiet=False, name='', additional_decoders=None, prob_baseline=None):
        """
        Decoder 초기화
        
        Args:
            formatter: 데이터 포맷터
            dataset: 평가 데이터셋
            n_guesses: 허용되는 최대 추측 횟수
            max_outputs: 최대 출력 개수
            frac_score: 분수 점수 사용 여부
            quiet: 조용한 모드 여부
            name: 디코더 이름
            additional_decoders: 추가 디코더들
            prob_baseline: 확률 기준선
        """
        self.formatter = formatter
        self.dataset = dataset
        self.n_guesses = n_guesses
        self.decoded_results = {}  # 디코딩된 결과들
        self.correct_solutions = {}  # 정답 솔루션들
        self.keys_lim = set()  # 제한된 추측 내에서 정답을 맞춘 키들
        self.keys_all = set()  # 모든 추측에서 정답을 맞춘 키들
        self.mult_cnt = {}  # 배수 카운트
        self.keys_cnt = {}  # 키 카운트
        self.frac_score = frac_score
        self.max_outputs = max_outputs
        self.quiet = quiet
        # 입력과 응답의 길이 정보 계산
        self.input_len = [{} if formatter is not None and formatter.tokenizer is None else ds.get_lengths(formatter, name='input') for ds in [dataset, dataset.mod(np.transpose, keep_key=True)]]
        self.reply_len = [{} if formatter is not None and formatter.tokenizer is None else ds.get_lengths(formatter, name='reply') for ds in [dataset, dataset.mod(np.transpose, keep_key=True)]]
        self.additional_decoders = additional_decoders
        self.name = name
        self.prob_tracker = {}  # 확률 추적기
        self.prob_tracker_best = {}  # 최고 확률 추적기
        self.prob_baseline = prob_baseline

    def score(self, *to_score):
        """
        점수를 계산하는 함수
        
        Args:
            *to_score: 점수를 매길 집합들
        
        Returns:
            (점수들, 총 개수) 튜플
        """
        scores = [(sum(1/self.mult_cnt[k.split('_')[0]] for k in s) if self.frac_score else len(s)) for s in to_score]
        score_cnt = len(self.mult_cnt if self.frac_score else self.keys_cnt)
        return scores, score_cnt

    def from_store(self, store, **kwargs):
        """
        저장소에서 결과를 로드하여 처리하는 함수
        
        Args:
            store: 저장소 경로
            **kwargs: 추가 인자들
        
        Returns:
            self 객체
        """
        for key, outputs in inference_load(store).items():
            self.process(key, outputs, **kwargs)
        return self

    def score_fmt(self, v):
        """점수 포맷팅 함수"""
        return f'{v:5.1f}' if self.frac_score else f'{v:3}'

    def process_single_output(self, key, output_len, decoded, print_func=print, len_info=None, device_info=None):
        """
        단일 출력을 처리하는 함수
        
        Args:
            key: 결과 키
            output_len: 출력 길이
            decoded: 디코딩된 결과
            print_func: 출력 함수
            len_info: 길이 정보
            device_info: 디바이스 정보
        """
        import numpy as np
        # 데이터셋 변환 역변환 적용
        inv_mod = {k: v if k.endswith('val') else self.dataset.invert_mod(v, key, inv_perm=(k.startswith('output') or k.startswith('score_all'))) for k, v in decoded.items()}
        base_key = key.split('.')[0]
        self.decoded_results[base_key] = self.decoded_results.get(base_key, {})
        self.decoded_results[base_key][key] = inv_mod
        output = inv_mod.get('output')
        score = inv_mod.get('score')

        # 빠른 점수 계산
        self.keys_cnt[base_key] = self.keys_cnt.get(base_key, 0) + 1
        mult_key, mult_sub = (base_key.split('_') + ['0'])[:2]
        self.mult_cnt[mult_key] = max(self.mult_cnt.get(mult_key, 0), int(mult_sub) + 1)
        
        if len(self.dataset.replies):
            correct_solution = self.dataset.replies.get(base_key)
            if correct_solution is not None:
                correct_solution = correct_solution[0]
                self.correct_solutions[base_key] = correct_solution
                is_correct = correct_solution is not None and np.array_equal(correct_solution, output)
                if is_correct:
                    self.keys_all.add(base_key)
                    if self.keys_cnt[base_key] <= self.n_guesses: self.keys_lim.add(base_key)
            
            # 정답 여부 문자열 생성
            corr_str = 'cant_decode' if output is None else 'sol_unknown' if correct_solution is None else 'ALL_CORRECT' if is_correct else 'bad_xy_size' if np.shape(correct_solution)!=np.shape(output) else 'bad_content'
            (score_lim, score_all), score_cnt = self.score(self.keys_lim, self.keys_all)

            tp_arr = (key.count('transpose') + key.count('rot90')) % 2
            msc = None if score is None else np.sum(score)
            fsc = inv_mod.get('score_val')
            
            # 확률 추적
            if output is not None and fsc is not None:
                pt = self.prob_tracker[base_key] = self.prob_tracker.get(base_key, {})
                hash = tuple(map(tuple, output))
                prob = pt[hash] = pt.get(hash, 0) + (np.exp(fsc) if self.prob_baseline is None else fsc - np.log(self.prob_baseline))
                current_best = self.prob_tracker_best.get(base_key)
                if current_best is None or current_best[0]<prob:
                    self.prob_tracker_best[base_key] = (prob, output)
            
            # 결과 출력 포맷팅
            fmt_name = f'{self.name}: ' if self.name else ''
            msc_print = f'{min(-msc, 9.99999):7.5f}' if msc is not None else 'unknown'
            fsc_print = f'{min(-fsc, 9.99999):7.5f}' if fsc is not None else 'unknown'
            if not self.quiet: 
                print_func(f" {fmt_name}acc: {self.score_fmt(score_lim)}/{score_cnt:3}={min(score_lim/score_cnt, 0.999):5.1%} (2-guess), {self.score_fmt(score_all)}/{score_cnt:3}={min(score_all/score_cnt, 0.999):5.1%} (any);{f' {device_info}' if device_info else ''} tok:{self.input_len[tp_arr].get(base_key, '?'):>4}+{self.reply_len[tp_arr].get(base_key, '?'):>3}>{'n/a' if output_len is None else output_len:>3} {corr_str}:{msc_print}|{fsc_print} [{key}]")

    def get_current_best(self, base_key):
        """
        현재 최고 결과를 가져오는 함수
        
        Args:
            base_key: 기본 키
        
        Returns:
            최고 결과 또는 None
        """
        current_best = self.prob_tracker_best.get(base_key)
        return None if current_best is None else current_best[1]

    def process_single_decode(self, key, de_tokenized, print_func=print, **kwargs):
        """
        단일 디코딩 결과를 처리하는 함수
        
        Args:
            key: 결과 키
            de_tokenized: 디토크나이즈된 결과
            print_func: 출력 함수
            **kwargs: 추가 인자들
        """
        # 호환성을 위한 포맷 확인
        if len(de_tokenized)==3 and not isinstance(de_tokenized[1], float):  
            output_len, *data = de_tokenized
            score_val = None
        else: 
            output_len, score_val, *data = de_tokenized
        
        if self.formatter is None:
            assert len(data) == 1
            decoded = [data[0]]
        else: 
            decoded = self.formatter.decode_to_array(*data)
        
        # 점수 값 추가
        for d in decoded: d['score_val'] = score_val
        
        # 각 디코딩 결과 처리
        for i, dec in enumerate(decoded):
            if i==0: 
                self.process_single_output(key, output_len, dec, print_func=print_func, **kwargs)
            elif self.additional_decoders:
                if i-1<len(self.additional_decoders): 
                    self.additional_decoders[i-1].process_single_output(key, output_len, dec, print_func=print_func, **kwargs)
                else: 
                    print_func(f'{key} 출력 #{i}에 사용할 수 있는 디코더가 없습니다')
            else: 
                self.process_single_output(f'{key}.fix{i}', output_len, dec, print_func=print_func, **kwargs)

    def process(self, key, de_tokenized, **kwargs):
        """
        디토크나이즈된 결과들을 처리하는 함수
        
        Args:
            key: 결과 키
            de_tokenized: 디토크나이즈된 결과들
            **kwargs: 추가 인자들
        """
        for i, d in enumerate(de_tokenized):
            if self.max_outputs is None or i<=self.max_outputs:
                self.process_single_decode(f'{key}.out{i}', d, **kwargs)

    def get_unsolved_keys(self):
        """
        아직 해결되지 않은 키들을 반환하는 함수
        
        Returns:
            해결되지 않은 키들의 리스트
        """
        unsolved = []
        for base_key, reply in self.dataset.replies.items():
            if not any(np.array_equal(reply[0], s.get('output')) for s in self.decoded_results.get(base_key, {}).values()):
                unsolved.append(base_key)
        return unsolved

    def run_selection_algo(self, selection_algorithm):
        """
        선택 알고리즘을 실행하는 함수
        
        Args:
            selection_algorithm: 선택 알고리즘 함수
        
        Returns:
            선택된 결과들의 딕셔너리
        """
        return {bk: (selection_algorithm({k: g for k, g in v.items() if g.get('output') is not None}) if any(g.get('output') is not None for g in v.values()) else []) for bk, v in self.decoded_results.items()}

    def benchmark_selection_algos(self, selection_algorithms, skip_failed=True):
        """
        선택 알고리즘들을 벤치마크하는 함수
        
        Args:
            selection_algorithms: 테스트할 선택 알고리즘들
            skip_failed: 실패한 알고리즘 건너뛸지 여부
        
        Returns:
            벤치마크 결과 딕셔너리
        """
        import numpy as np
        results = {}
        print('*** 선택 알고리즘 벤치마크 중...')
        for selection_algorithm in selection_algorithms:
            name = selection_algorithm.__name__
            try:
                selected = self.run_selection_algo(selection_algorithm)
                if self.formatter is not None:
                    for sols in selected.values():
                        for s in sols:
                            assert self.formatter.is_valid_solution(s), f'유효하지 않은 솔루션 발견 {s}'
                correct_keys = {k for k, v in selected.items() if self.correct_solutions.get(k) is not None and any(np.array_equal(guess, self.correct_solutions[k]) for guess in v[:self.n_guesses])}
                (score,), score_cnt = self.score(correct_keys)
                results[name] = score
                print(f" acc: {score:5.1f}/{score_cnt:3}={score/score_cnt:6.2%} ('{name}')")
            except:
                print(f" {'실행 실패':>21} ('{name}')")
                if not skip_failed: raise
        return results

    def calc_augmented_scores(self, model, base_keys=None, store=None, seed=0, max_len=None, make_unique=True, quiet=False, **kwargs):
        """
        증강된 점수를 계산하는 함수
        
        Args:
            model: 평가할 모델
            base_keys: 기본 키들 (None이면 모든 키)
            store: 저장 경로
            seed: 랜덤 시드
            max_len: 최대 길이
            make_unique: 고유성 확보 여부
            quiet: 조용한 모드 여부
            **kwargs: 추가 인자들
        """
        if base_keys is None: base_keys = list(self.decoded_results.keys())
        if store is not None: store = f'{store}_new'  # 새 포맷은 하위 호환되지 않으므로 새 폴더 사용
        
        for bk in (base_keys if quiet else tqdm(base_keys, desc='증강된 점수 계산', file=sys.stdout)):
            res = self.decoded_results.get(bk, {})
            known_scores = {}
            for k, v in sorted(res.items()):
                if 'output' in v:
                    k_store = None if store is None else os.path.join(store, k)
                    id = tuple(map(tuple, v['output']))
                    if not (make_unique and id in known_scores):
                        try:
                            assert k_store is not None
                            with bz2.BZ2File(k_store) as f: 
                                known_scores[id] = pickle.load(f)
                            # 하위 호환성을 위한 포맷 변환
                            if isinstance(known_scores[id], list): 
                                known_scores[id] = dict(score_multi=known_scores[id])  
                            k_store = None
                        except:
                            # 임시 데이터셋 생성하여 점수 계산
                            temp_dataset = self.dataset.__class__(
                                keys=[bk],
                                queries={bk: self.dataset.queries.get(bk)},
                                replies={bk: [v['output'].tolist()]},
                            )
                            temp_decoder = self.__class__(self.formatter, temp_dataset, n_guesses=self.n_guesses, quiet=True)
                            temp_dataset = temp_dataset.augment(**kwargs, seed=(seed+hash(k)+hash(id)) % 1024**2, quiet=True)
                            if max_len is not None: 
                                temp_dataset = temp_dataset.cut_to_len(formatter=self.formatter, name='input', max_len=max_len, quiet=True)
                            for x in temp_dataset.as_list(self.formatter): 
                                calc_score(**x, formatter=self.formatter, model=model, decoder=temp_decoder)
                            
                            # 다양한 점수 메트릭 저장
                            known_scores[id] = dict(
                                score_multi=[np.sum(x['score']) for x in temp_decoder.decoded_results[bk].values()],
                                score_multi_nl=[x['score_val'] for x in temp_decoder.decoded_results[bk].values()],
                                score_multi_array=np.array([x['score'] for x in temp_decoder.decoded_results[bk].values()]),
                                score_multi_array_cum=np.array([x['score_cum'] for x in temp_decoder.decoded_results[bk].values()]),
                                score_multi_array_all=np.array([x['score_all'] for x in temp_decoder.decoded_results[bk].values()]),
                                score_multi_array_all_cum=np.array([x['score_all_cum'] for x in temp_decoder.decoded_results[bk].values()]),
                            )
                            if k_store is not None:
                                os.makedirs(store, exist_ok=True)
                                with bz2.BZ2File(k_store, 'w') as f: 
                                    pickle.dump(known_scores[id], f)
                    v.update(known_scores[id])

def turbo_dfs(model, logits, path, eos_token_id, max_new_tokens, max_score, max_score_greedy, temperature, suppress_tokens, torch, score=0.0, pos=0, cache=None):
    """
    터보 깊이 우선 탐색 함수 (효율적인 빔 서치 대안)
    
    Args:
        model: 언어 모델
        logits: 로짓 값들
        path: 미리 계산된 경로
        eos_token_id: 문장 종료 토큰 ID
        max_new_tokens: 최대 새 토큰 수
        max_score: 최대 점수
        max_score_greedy: 탐욕적 최대 점수
        temperature: 온도 파라미터
        suppress_tokens: 억제할 토큰들
        torch: PyTorch 모듈
        score: 현재 점수
        pos: 현재 위치
        cache: 캐시
    
    Returns:
        (점수, 접미사, 로짓들) 튜플들의 리스트
    """
    logits, next_logits = logits[0], (logits[1:] if len(logits)>1 else None)
    nll = -(logits / temperature).detach().float().log_softmax(-1).cpu().numpy()
    greedy_index = nll.argmin(-1).item()
    nll = list(enumerate(nll))
    
    # 미리 계산된 경로가 있으면 먼저 따라가기
    if path: nll[0], nll[path[0]], path = nll[path[0]], nll[0], path[1:]  
    
    suffixes = []
    for i, s in nll:
        next_score = score + s
        allowed_max_score = max_score_greedy if i==greedy_index else max_score
        if next_score < allowed_max_score:
            if i==eos_token_id: 
                next_suffixes = [(next_score, [], [])]
            elif max_new_tokens>1:
                if next_logits is None:
                    # 캐시 크기 조정
                    if pos<cache[0][0][0].shape[2]: 
                        cache[0] = tuple(tuple(c[:, :, :pos] for c in l) for l in cache[0])
                    # 다음 토큰 생성
                    next_logits, cache[0] = model(
                        input_ids= torch.full((1,1), i, device=model.device),
                        position_ids=torch.full((1,1), pos, device=model.device),
                        past_key_values=cache[0],
                    )[:2]
                    next_logits = next_logits[0]  # 배치 차원 제거
                # 재귀 호출
                next_suffixes = turbo_dfs(model, logits=next_logits, path=path, eos_token_id=eos_token_id, max_new_tokens=max_new_tokens-1, max_score=max_score, max_score_greedy=allowed_max_score, temperature=temperature, suppress_tokens=suppress_tokens, torch=torch, score=next_score, pos=pos+1, cache=cache)
            else: 
                next_suffixes = []
            
            # 접미사에 현재 토큰과 로짓 추가
            for suffix in next_suffixes:
                suffix[1].append(i)
                suffix[2].append(logits)
            suffixes.extend(next_suffixes)
        next_logits = None
    return suffixes

def inference_turbo_dfs(model, input_ids, eos_token_id, max_new_tokens, min_prob, min_prob_greedy=1, temperature=1.0, suppress_tokens=[], path=[], attention_mask=None):
    """
    터보 DFS를 사용한 추론 함수
    
    Args:
        model: 언어 모델
        input_ids: 입력 토큰 ID들
        eos_token_id: 문장 종료 토큰 ID
        max_new_tokens: 최대 새 토큰 수
        min_prob: 최소 확률
        min_prob_greedy: 탐욕적 최소 확률
        temperature: 온도 파라미터
        suppress_tokens: 억제할 토큰들
        path: 경로
        attention_mask: 어텐션 마스크
    
    Returns:
        정렬된 결과 리스트 (점수, 접미사, 점수 배열)
    """
    import torch
    with torch.no_grad():
        assert attention_mask is None or attention_mask.all(), '구현되지 않음'
        input_ids = torch.as_tensor(input_ids, device=model.device, dtype=int)
        if input_ids.ndim==2: input_ids = input_ids.squeeze(0)
        assert input_ids.ndim==1, '배칭은 지원되지 않음'
        
        # 점수 임계값 계산
        max_score = -np.log(min_prob)
        max_score_greedy = (-np.log(min_prob_greedy)) if min_prob_greedy>0 else float('inf')  
        max_score_greedy = max(max_score, max_score_greedy)
        
        if path is None: path = []
        if len(path) and path[-1]==eos_token_id: path = path[:-1]
        
        with torch.no_grad():
            full_path = input_ids
            if len(path): 
                full_path = torch.cat([full_path, torch.as_tensor(path, device=model.device)])
            logits, cache = model(input_ids=full_path[np.newaxis])[:2]
            logits = logits[0, len(input_ids)-1:]
        
        # 터보 DFS 실행
        result = turbo_dfs(model, logits=logits, path=path, eos_token_id=eos_token_id, max_new_tokens=max_new_tokens, max_score=max_score, max_score_greedy=max_score_greedy, temperature=temperature, suppress_tokens=suppress_tokens, torch=torch, score=0.0, pos=len(input_ids), cache=[cache])
        
        # 결과 정렬하여 반환
        return sorted([(score_val, np.array(suffix[::-1]), torch.stack(score_arr[::-1]).float().cpu().numpy()) for score_val, suffix, score_arr in result], key=lambda x:x[0])

def inference_step(tokenized, model, remove_token_type_ids=True, num_beams=1, formatter=None, min_prob=None, current_best=None, **kwargs):
    """
    추론 단계를 실행하는 함수
    
    Args:
        tokenized: 토크나이즈된 입력
        model: 언어 모델
        remove_token_type_ids: 토큰 타입 ID 제거 여부
        num_beams: 빔 개수
        formatter: 포맷터
        min_prob: 최소 확률
        current_best: 현재 최고 결과
        **kwargs: 추가 인자들
    
    Returns:
        (토큰 출력, 점수 출력) 튜플
    """
    import torch
    if remove_token_type_ids: tokenized.pop('token_type_ids', None)
    
    if min_prob is not None:
        assert num_beams==1
        # 터보 DFS 사용
        gen = inference_turbo_dfs(model, **tokenized.to(model.device), path=current_best, min_prob=min_prob, eos_token_id=formatter.tokenizer.eos_token_id, **kwargs)
        tokens_out = [[g[1] for g in gen]]
        scores_out = [[g[2] for g in gen]]
    elif is_unsloth_model(model) and num_beams > 1:
        assert False, 'unsloth는 빔 서치를 지원하지 않습니다'
    else:
        # 표준 생성 방식
        gen = model.generate(**tokenized.to(model.device), return_dict_in_generate=True, output_logits=True, use_cache=True, **kwargs)
        tokens_out = gen['sequences'][:, torch.newaxis, tokenized['input_ids'].shape[-1]:].cpu().numpy().copy()
        scores_out = torch.stack(gen['logits'], axis=-2)[:, torch.newaxis].float().cpu().numpy().copy()
    return tokens_out, scores_out

def process_inference_output(key, outputs, formatter, store=None, decoder=None, decoder_args={}):
    """
    추론 출력을 처리하는 함수
    
    Args:
        key: 결과 키
        outputs: 출력 결과들
        formatter: 포맷터
        store: 저장 경로
        decoder: 디코더
        decoder_args: 디코더 인자들
    
    Returns:
        디토크나이즈된 결과
    """
    de_tokenized = [formatter.de_tokenize(*output) for output in zip(*outputs)]
    inference_save(store, key, de_tokenized)
    if decoder is not None: decoder.process(key, de_tokenized, **decoder_args)
    return de_tokenized

def inference_run_v2(model, formatter, dataset, decoder=None, max_new_tokens=None, max_batch_size=1, store=None, result_dict=None, rerun_empty=False, retrain=None, use_turbo=False, group_multi_output=True, **kwargs):
    """
    추론 실행 함수 (버전 2)
    
    Args:
        model: 언어 모델
        formatter: 데이터 포맷터
        dataset: 데이터셋
        decoder: 디코더
        max_new_tokens: 최대 새 토큰 수
        max_batch_size: 최대 배치 크기
        store: 저장 경로
        result_dict: 결과 딕셔너리
        rerun_empty: 빈 결과 재실행 여부
        retrain: 재훈련 함수
        use_turbo: 터보 모드 사용 여부
        group_multi_output: 다중 출력 그룹화 여부
        **kwargs: 추가 인자들
    
    Returns:
        결과 딕셔너리
    """
    import torch
    assert max_batch_size==1, '지원되지 않음'

    with torch.no_grad():
        print('*** 저장된 데이터 로드 중...')
        if result_dict is None: result_dict = {}
        result_dict = inference_load(store, dataset.keys, result_dict)
        
        # 키들을 기본 키별로 그룹화
        by_base_key = {}
        needs_rerun = {}
        base_key_list = []
        for key in dataset.keys:
            base_key = key.split('.')[0]
            if group_multi_output: base_key = base_key.split('_')[0]
            if base_key not in by_base_key: base_key_list.append(base_key)
            bk_list = by_base_key[base_key] = by_base_key.get(base_key, [])
            bk_list.append(key)
        
        # 재실행이 필요한 키들 찾기
        for base_key, keys in by_base_key.items():
            for key in keys:
                de_tokenized = result_dict.get(key)
                if de_tokenized is None or (rerun_empty and not de_tokenized):
                    bk_list = needs_rerun[base_key] = needs_rerun.get(base_key, [])
                    bk_list.append(key)
                elif decoder is not None: 
                    decoder.process(key, de_tokenized)

        # 모델을 추론 모드로 설정
        formatter.tokenizer.padding_side = 'left'
        if max_new_tokens is None: max_new_tokens = formatter.max_new_tokens()
        if is_unsloth_model(model):
            from unsloth import FastLanguageModel
            FastLanguageModel.for_inference(model)
        else: 
            model.eval()

        print('*** 추론 실행 시작...')
    try:
        with tqdm(base_key_list, file=sys.stdout) as pbar:
            for base_key in pbar:
                run_keys = needs_rerun.get(base_key)
                if run_keys:
                    # 재훈련이 필요한 경우
                    if retrain is not None:
                        retrain_dataset = dataset.keep_key_startswith(base_key)
                        print(f"키 '{base_key}'에 대해 모델 재훈련 중 (retrain_dataset_size={len(retrain_dataset.keys)})")
                        retrain(model, retrain_dataset)
                        if is_unsloth_model(model): FastLanguageModel.for_inference(model)
                    
                    with torch.no_grad():
                        for key in run_keys:
                            # 입력 텍스트 준비
                            input_text = dataset.get(key, formatter)['input']
                            batch = formatter.tokenizer([input_text], return_tensors='pt')
                            
                            # 터보 모드에서 현재 최고 결과 사용
                            current_best = decoder.get_current_best(key.split('.')[0]) if use_turbo else None
                            if current_best is not None:
                                current_best = dataset.forward_mod(current_best, key)
                                current_best = formatter.fmt_reply([current_best])
                                current_best = formatter.tokenizer(input_text+current_best)['input_ids'][batch['input_ids'].shape[-1]:]
                            
                            # 추론 실행
                            batch_out = inference_step(batch, model, formatter=formatter, max_new_tokens=max_new_tokens, current_best=current_best, **kwargs)
                            outputs = [x[0] for x in batch_out]
                            result_dict[key] = process_inference_output(key, outputs, formatter, store=store, decoder=decoder, decoder_args=dict(print_func=pbar.write))
        print('*** 추론 실행 완료.')
    except KeyboardInterrupt: 
        print('*** Ctrl+C 눌림, 추론 실행 중단.')
    return result_dict

class Retrainer(object):
    """
    모델 재훈련을 담당하는 클래스
    """
    def __init__(self, n, aug_opts, reload_state_dict=None, **kwargs):
        """
        Retrainer 초기화
        
        Args:
            n: 훈련 샘플 수
            aug_opts: 데이터 증강 옵션들
            reload_state_dict: 재로드할 state_dict
            **kwargs: 추가 인자들
        """
        self.n = n
        self.aug_opts = aug_opts
        self.reload_state_dict = reload_state_dict
        self.kwargs = kwargs

    def preprocess(self, dataset):
        """
        데이터셋을 전처리하는 함수
        
        Args:
            dataset: 입력 데이터셋
        
        Returns:
            전처리된 데이터셋
        """
        # 필요한 수만큼 데이터 증강
        ds = [dataset.augment(quiet=True, shfl_keys=True, **self.aug_opts) for _ in range((self.n-1)//dataset.length()+1)]
        ds = ds[0] if len(ds)==1 else ds[0].append(*ds[1:])
        ds, _ = ds.split_at_pos(self.n)
        return ds

    def __call__(self, model, dataset):
        """
        재훈련 실행
        
        Args:
            model: 재훈련할 모델
            dataset: 훈련 데이터셋
        """
        if self.reload_state_dict is not None: 
            set_peft_weights(model, self.reload_state_dict)
        
        assert is_unsloth_model(model), '구현되지 않음'
        if is_unsloth_model(model):
            from unsloth import FastLanguageModel
            FastLanguageModel.for_training(model)
        else: 
            model.train()
        
        training_run(model, dataset=self.preprocess(dataset), **self.kwargs)

def calc_score(key, input, reply, formatter, model, store=None, decoder=None, **_):
    """
    주어진 입력-응답 쌍에 대한 점수를 계산하는 함수
    
    Args:
        key: 데이터 키
        input: 입력 텍스트
        reply: 응답 텍스트
        formatter: 데이터 포맷터
        model: 언어 모델
        store: 저장 경로
        decoder: 디코더
        **_: 무시되는 추가 인자들
    """
    import torch
    with torch.no_grad():
        # 입력 길이 계산
        input_len = len(formatter.tokenizer(input)['input_ids'])
        # 전체 시퀀스 토크나이즈
        tokenized = formatter.tokenizer([input+reply], return_tensors='pt')
        # 응답 부분의 토큰들만 추출
        reply_tok = tokenized['input_ids'][0][input_len:].cpu().numpy().copy()
        # 응답 부분의 로그 확률 계산
        reply_log = model.forward(**tokenized.to(model.device))['logits'][0, input_len-1: -1].float().cpu().numpy().copy()
        # 결과 처리
        process_inference_output(key, (reply_tok[torch.newaxis], reply_log[torch.newaxis]), formatter, store=store, decoder=decoder)

def mem_info(gpu_id=0):
    """
    GPU 메모리 정보를 출력하는 함수
    
    Args:
        gpu_id: GPU ID (기본값: 0)
    """
    import torch
    try:
        gpu_stats = torch.cuda.get_device_properties(gpu_id)
        usage = torch.cuda.max_memory_reserved() / 1024**3
        avail = gpu_stats.total_memory / 1024**3
        print(f"*** GPU: {gpu_stats.name}, 사용량 {usage:.3} / {avail:.3} GB.")
    except: 
        print('*** 메모리 통계를 가져오는 중 예외가 발생했습니다.')

In [ ]:
%%writefile arc_loader.py
import json
import numpy as np
import hashlib
import os, sys
from tqdm import tqdm
from glob import glob
import itertools
import random

def cut_at_token(output, token_id):
    """
    특정 토큰에서 출력을 자르는 함수
    
    Args:
        output: 출력 시퀀스
        token_id: 자를 토큰 ID
    
    Returns:
        자른 출력 시퀀스
    """
    eos_positions = (output==token_id).nonzero()[0]
    return output[:eos_positions[0]] if len(eos_positions) else output

def shuffled(data_list):
    """데이터 리스트를 무작위로 섞는 함수"""
    return np.random.permutation(data_list).tolist()

def permute_mod(a, descriptor, invert=False):
    """
    배열에 순열 변환을 적용하는 함수
    
    Args:
        a: 변환할 배열
        descriptor: 순열을 나타내는 문자열
        invert: 역변환 여부
    
    Returns:
        순열이 적용된 배열
    """
    permutation = [int(i) for i in descriptor if str(i).isdigit()]
    assert sorted(permutation)==list(range(10))
    a = np.asarray(a)
    if a.ndim==3:
        # 3차원 배열의 경우 (색상 차원)
        if not invert: permutation = np.argsort(permutation)
        a = a[..., permutation]
    else:
        # 2차원 배열의 경우 (그리드)
        assert a.ndim==2
        if invert: permutation = np.argsort(permutation)
        a = np.asarray(permutation)[a]
    return a

def permute_rnd_col_(query):
    """배경색(0)을 유지하고 나머지 색상을 무작위로 순열하는 함수"""
    permutation = [0]+(1+np.random.permutation(9)).tolist()
    return 'permute' + ''.join(map(str, permutation))

def permute_rnd_all_(query):
    """모든 색상을 무작위로 순열하는 함수"""
    permutation = np.random.permutation(10).tolist()
    return 'permute' + ''.join(map(str, permutation))

def permute_cnt_col_(query):
    """
    배경색을 유지하고 나머지 색상을 빈도순으로 정렬하는 함수
    (무작위성을 동점자 해결 기준으로 사용)
    """
    elements, frequency = np.unique(np.concatenate([list(range(10))]+[np.array(x['input']).ravel() for x in query['train']]), return_counts=True)
    permutation = [0]+sorted(np.random.permutation(9)+1, key=lambda i: frequency[i], reverse=True)  
    return 'permute' + ''.join(map(str, permutation))

def permute_cnt_all_(query):
    """모든 색상을 빈도순으로 정렬하는 함수 (무작위성을 동점자 해결 기준으로 사용)"""
    elements, frequency = np.unique(np.concatenate([list(range(10))]+[np.array(x['input']).ravel() for x in query['train']]), return_counts=True)
    permutation = sorted(np.random.permutation(10), key=lambda i: frequency[i], reverse=True)  
    return 'permute' + ''.join(map(str, permutation))

# 다양한 순열 변환 옵션들
permute_rnd_col = (permute_mod, permute_rnd_col_)  # 배경색 유지, 무작위 순열
permute_rnd_all = (permute_mod, permute_rnd_all_)  # 전체 무작위 순열
permute_cnt_col = (permute_mod, permute_cnt_col_)  # 배경색 유지, 빈도순 정렬
permute_cnt_all = (permute_mod, permute_cnt_all_)  # 전체 빈도순 정렬
permute_None = (np.copy, None)  # 순열 없음 (복사만)

class ArcDataset(object):
    """ARC 데이터셋을 처리하는 메인 클래스"""
    
    @staticmethod
    def forward_mod(a, key, use_perm=True, is_output=True):
        """
        키에 따라 배열에 순방향 변환을 적용하는 함수
        
        Args:
            a: 변환할 배열
            key: 변환 정보가 포함된 키
            use_perm: 순열 사용 여부
            is_output: 출력 데이터 여부
        
        Returns:
            변환된 배열
        """
        if a is None: return a
        for op in key.split('.')[1:]:
            # 'I' 접두사는 입력에만 적용되는 변환을 의미
            if op.startswith('I'):
                if is_output: continue
                op = op[1:]
            # 다양한 변환 적용
            if   op=='rot90':              a = np.rot90(a)  # 90도 회전
            elif op=='transpose':          a = np.swapaxes(a, 0, 1)  # 전치
            elif op.startswith('permute'): a = permute_mod(a, op, invert=False) if use_perm else a  # 순열
            elif op.startswith('copy'):    a = np.copy(a)  # 복사
            elif op.startswith('out'):     a = a  # 출력 표시
            elif op.startswith('ex'):      a = a  # 예제 표시
            elif op.startswith('fix'):     a = a  # 수정 표시
            elif op.startswith('ice'):     a = a  # icecuber 솔루션 추가용
            else: raise NotImplementedError(f"연산 '{op}'의 역변환을 알 수 없습니다.")
        return a

    @staticmethod
    def invert_mod(a, key, inv_perm=True, is_output=True):
        """
        키에 따라 배열에 역방향 변환을 적용하는 함수
        
        Args:
            a: 변환할 배열
            key: 변환 정보가 포함된 키
            inv_perm: 순열 역변환 사용 여부
            is_output: 출력 데이터 여부
        
        Returns:
            역변환된 배열
        """
        if a is None: return a
        # 변환을 역순으로 적용
        for op in key.split('.')[1:][::-1]:
            if op.startswith('I'):
                if is_output: continue
                op = op[1:]
            # 역변환 적용
            if   op=='rot90':              a = np.rot90(np.rot90(np.rot90(a)))  # 270도 회전 (90도의 역)
            elif op=='transpose':          a = np.swapaxes(a, 0, 1)  # 전치 (자기 자신이 역변환)
            elif op.startswith('permute'): a = permute_mod(a, op, invert=True) if inv_perm else a
            elif op.startswith('copy'):    a = np.copy(a)
            elif op.startswith('out'):     a = a
            elif op.startswith('ex'):      a = a
            elif op.startswith('fix'):     a = a
            elif op.startswith('ice'):     a = a
            else: raise NotImplementedError(f"연산 '{op}'의 역변환을 알 수 없습니다.")
        return a

    def __init__(self, queries, replies={}, keys=None, is_orig=False, is_fake=False):
        """
        ArcDataset 초기화
        
        Args:
            queries: 문제 데이터 딕셔너리
            replies: 답안 데이터 딕셔너리
            keys: 사용할 키 리스트
            is_orig: 원본 데이터셋 여부
            is_fake: 가짜 테스트 세트 여부
        """
        if keys is not None: keys = [k for k in keys if k is not None]
        self.queries = queries if keys is None else {k: queries[k] for k in keys}
        self.replies = replies if keys is None else {k: replies[k] for k in keys if k in replies}
        self.is_orig = is_orig
        self.is_fake = is_fake
        self.keys = sorted(queries.keys()) if keys is None else keys
        self.faulty = {}  # 결함이 있는 데이터 추적
        self.transposed_dataset = None  # 전치된 데이터셋 캐시

    @classmethod
    def empty(cls):
        """빈 데이터셋을 생성하는 클래스 메서드"""
        return cls(queries={}, replies={}, keys=[])

    def change_keys(self, keys, keep_flags=False):
        """
        키를 변경하여 새로운 데이터셋을 생성하는 함수
        
        Args:
            keys: 새로운 키 리스트
            keep_flags: 플래그 유지 여부
        
        Returns:
            새로운 ArcDataset 인스턴스
        """
        flags = dict(is_fake=self.is_fake, is_orig=self.is_orig) if keep_flags else {}
        return self.__class__(queries=self.queries, replies=self.replies, keys=keys, **flags)

    @classmethod
    def from_file(cls, queries_file):
        """
        파일에서 문제 데이터를 로드하는 클래스 메서드
        
        Args:
            queries_file: 문제 파일 경로
        
        Returns:
            로드된 ArcDataset 인스턴스
        """
        print(f"*** '{queries_file}'에서 문제 로드 중...")
        with open(queries_file) as f: queries = f.read()
        # 가짜 테스트 세트 감지 (특정 MD5 해시로 판별)
        is_fake = hashlib.md5(queries.encode('utf-8')).hexdigest().lower()=='a6b7dac3cab03abf2eb333e16610d6dc'
        if is_fake: print("*** -> 가짜 테스트 세트 감지됨, 'is_fake' 플래그를 True로 설정.")
        return cls(
            queries=json.loads(queries),
            is_fake=is_fake,
            is_orig=True,
        )

    def load_replies(self, replies_file):
        """
        답안 파일을 로드하는 함수
        
        Args:
            replies_file: 답안 파일 경로
        
        Returns:
            self 객체
        """
        print(f"*** '{replies_file}'에서 솔루션 로드 중...")
        with open(replies_file) as f: replies = f.read()
        replies_parsed = json.loads(replies)
        self.replies = {k: replies_parsed[k] for k in self.keys}
        return self

    def split_multi_replies(self):
        """
        다중 테스트 케이스를 개별 키로 분할하는 함수
        
        Returns:
            분할된 새로운 ArcDataset
        """
        key_indices = [(k, i) for k in self.keys for i in range(len(self.queries[k]['test']))]
        return self.__class__(
            keys=[f'{k}_{i}' for k, i in key_indices],
            queries={f'{k}_{i}': {'train': self.queries[k]['train'], 'test': [self.queries[k]['test'][i]]} for k, i in key_indices},
            replies={f'{k}_{i}': [self.replies[k][i]] for k, i in key_indices if k in self.replies},
        )

    def move_test_to_train(self):
        """
        테스트 데이터를 훈련 데이터로 이동하는 함수
        
        Returns:
            변환된 새로운 ArcDataset
        """
        new_queries = {k: {'train': self.queries[k]['train'] + [{**t, 'output': self.replies[k][i]} for i, t in enumerate(self.queries[k]['test'])], 'test': []} for k in self.keys}
        return self.__class__(queries=new_queries, keys=[k for k in self.keys])

    def last_train_ex_for_test(self):
        """
        마지막 훈련 예제를 테스트로 사용하는 함수
        
        Returns:
            변환된 새로운 ArcDataset
        """
        assert not self.replies
        new_queries = {k: {'train': self.queries[k]['train'][:-1], 'test': [{'input': self.queries[k]['train'][-1]['input']}]} for k in self.keys}
        new_replies = {k: [self.queries[k]['train'][-1]['output']] for k in self.keys}
        return self.__class__(queries=new_queries, replies=new_replies, keys=[k for k in self.keys])

    def length(self):
        """데이터셋의 길이를 반환하는 함수"""
        return len(self.keys)

    def shuffled(self, seed=None):
        """
        키를 무작위로 섞은 새로운 데이터셋을 반환하는 함수
        
        Args:
            seed: 랜덤 시드
        
        Returns:
            섞인 새로운 ArcDataset
        """
        if seed is not None: np.random.seed(seed)
        return self.__class__(queries=self.queries, replies=self.replies, keys=shuffled(self.keys))

    def sorted(self, **kwargs):
        """키를 정렬한 새로운 데이터셋을 반환하는 함수"""
        return self.__class__(queries=self.queries, replies=self.replies, keys=sorted(self.keys, **kwargs))

    def append(*datasets):
        """
        여러 데이터셋을 결합하는 정적 메서드
        
        Args:
            *datasets: 결합할 데이터셋들
        
        Returns:
            결합된 새로운 ArcDataset
        """
        return datasets[0].__class__(
            queries={k: v for d in datasets for k, v in d.queries.items()},
            replies={k: v for d in datasets for k, v in d.replies.items()},
            keys   =[k    for d in datasets for k    in d.keys           ],
        )

    def sort_ex_by_input_size(self, seed=42, reverse=False):
        """
        예제를 입력 크기순으로 정렬하는 함수
        
        Args:
            seed: 랜덤 시드
            reverse: 역순 정렬 여부
        
        Returns:
            정렬된 새로운 ArcDataset
        """
        np.random.seed(seed)
        sort_key = lambda ex: np.prod(np.shape(ex['input']))
        new_queries = {k2: {k: (sorted(np.random.permutation(np.array(v, dtype=object)), key=sort_key, reverse=reverse) if k=='train' else v) for k, v in v2.items()} for k2, v2 in self.queries.items()}
        return self.__class__(queries=new_queries, replies=self.replies, keys=[k for k in self.keys])

    def interleave(self, block_size, num_gpus=None):
        """
        데이터를 인터리브하여 분산 처리를 위해 분할하는 함수
        
        Args:
            block_size: 블록 크기
            num_gpus: GPU 개수
        
        Returns:
            인터리브된 데이터셋 또는 GPU별 데이터셋 리스트
        """
        keys = np.reshape(self.keys, (-1, block_size)).T
        if num_gpus is None: return self.change_keys(keys.ravel().tolist())
        ret, num_gpus = (None, num_gpus) if isinstance(num_gpus, int) else num_gpus
        keys = np.concatenate([keys, np.full((-keys.shape[0]%num_gpus, keys.shape[1]), None)])
        keys = np.reshape(keys, (keys.shape[0]//num_gpus, num_gpus, -1)).swapaxes(0, 1).reshape(num_gpus, -1)
        new_datasets = [self.change_keys(gpu_keys.tolist()) for gpu_keys in keys]
        return new_datasets if ret is None else new_datasets[ret]

    def remove(self, *datasets):
        """
        지정된 데이터셋의 키들을 제거하는 함수
        
        Args:
            *datasets: 제거할 데이터셋들
        
        Returns:
            키가 제거된 새로운 ArcDataset
        """
        remove_keys = {k for d in datasets for k in d.keys}
        new_keys = [k for k in self.keys if k not in remove_keys]
        return self.change_keys(new_keys)

    def keep_key_startswith(self, key_start):
        """
        특정 접두사로 시작하는 키만 유지하는 함수
        
        Args:
            key_start: 키 접두사
        
        Returns:
            필터링된 새로운 ArcDataset
        """
        new_keys = [k for k in self.keys if k.startswith(key_start)]
        return self.change_keys(new_keys)

    def mod_single(self, mod_func, descriptor, i, keep_key, inputs_only):
        """
        단일 변환을 적용하는 함수
        
        Args:
            mod_func: 변환 함수
            descriptor: 변환 설명자
            i: 인덱스
            keep_key: 키 유지 여부
            inputs_only: 입력만 변환할지 여부
        
        Returns:
            변환된 새로운 ArcDataset
        """
        queries = {}
        replies = {}
        keys    = []
        for k0 in self.keys:
            # 변환 설명자 생성
            desc = (('copy{i}' if mod_func is np.copy else mod_func.__name__) if descriptor is None else descriptor if isinstance(descriptor, str) else descriptor(self.queries[k0])).format(i=i)
            func = lambda a, d: np.asarray(mod_func(a) if descriptor is None else mod_func(a, d)).tolist()
            k1 = k0 if keep_key else f"{k0}.{'I' if inputs_only else ''}{desc}"
            keys.append(k1)
            # 쿼리 변환
            queries[k1] = {m: [{t: (func(a, desc) if t=='input' or not inputs_only else a) for t, a in x.items()} for x in e] for m, e in self.queries[k0].items()}
            # 답안 변환 (입력만 변환하는 경우가 아닐 때)
            if k0 in self.replies:
                replies[k1] = [func(a, desc) for a in self.replies[k0]]
        ret = self.__class__(queries=queries, replies=replies, keys=keys)
        return ret

    def mod(self, mod_func, descriptor=None, n=1, stack=None, keep=False, keep_key=False, shuffle=False, join=True, inputs_only=False):
        """
        데이터셋에 변환을 적용하는 메인 함수
        
        Args:
            mod_func: 변환 함수
            descriptor: 변환 설명자
            n: 변환 횟수
            stack: 스택 여부 (None이면 자동 결정)
            keep: 원본 유지 여부
            keep_key: 키 유지 여부
            shuffle: 셞플 여부
            join: 결합 여부
            inputs_only: 입력만 변환할지 여부
        
        Returns:
            변환된 ArcDataset (또는 데이터셋 리스트)
        """
        assert not (keep and keep_key)
        cur = self
        ret = [cur.shuffled() if shuffle else cur] if keep else []
        if stack is None: stack = mod_func.__name__.startswith('rot')  # 회전의 경우 기본적으로 스택
        for i in range(n):
            cur = (cur if stack else self).mod_single(mod_func, descriptor, i=i, keep_key=keep_key, inputs_only=inputs_only)
            ret.append(cur.shuffled() if shuffle else cur)
        return self.__class__.append(*ret) if join else ret

    def get(self, key, formatter):
        """
        특정 키의 데이터를 포맷된 형태로 가져오는 함수
        
        Args:
            key: 데이터 키
            formatter: 포맷터 객체
        
        Returns:
            포맷된 데이터 딕셔너리
        """
        assert formatter.out2_token is None or key in self.replies
        train = formatter.fmt_train(self.queries[key]['train'])
        query = formatter.fmt_query(self.queries[key]['test'], i=len(self.queries[key]['train']))
        reply = formatter.fmt_reply(self.replies[key], self.faulty.get(key)) if key in self.replies else ''
        text = train+query+reply if reply else formatter.fmt_train(self.queries[key]['train'], last_is_challenge=True)
        return dict(key=key, train=train, query=query, reply=reply, input=train+query, text=text)

    def as_list(self, formatter):
        """
        전체 데이터셋을 리스트 형태로 반환하는 함수
        
        Args:
            formatter: 포맷터 객체
        
        Returns:
            포맷된 데이터 리스트
        """
        return [self.get(key, formatter) for key in self.keys]

    def as_dataset(self):
        """HuggingFace Dataset 형태로 변환하는 함수"""
        from datasets import Dataset
        return Dataset.from_list([{'key': k, 'query': self.queries[k], 'reply': self.replies[k]} for k in self.keys])

    def get_length(self, key, formatter, name, max_of_transposed=False):
        """
        특정 키의 데이터 길이를 계산하는 함수
        
        Args:
            key: 데이터 키
            formatter: 포맷터 객체
            name: 계산할 부분 ('input' 또는 'reply')
            max_of_transposed: 전치된 버전과의 최대값 사용 여부
        
        Returns:
            데이터 길이
        """
        if formatter is None:
            # 포맷터가 없는 경우 원시 크기 계산
            if   name=='input': return sum(np.prod(np.shape(v)) for v3 in self.queries[key].values() for v2 in v3 for v in v2.values())
            elif name=='reply': return sum(np.prod(np.shape(v)) for v in self.replies[key])
            else: assert False
        else:
            # 포맷터를 사용한 토큰 길이 계산
            datasets = [self]
            if max_of_transposed:
                if self.transposed_dataset is None: self.transposed_dataset = self.mod(np.transpose, keep=False, keep_key=True)
                datasets.append(self.transposed_dataset)
            return max(len(formatter.tokenizer(ds.get(key, formatter=formatter)[name])['input_ids']) for ds in datasets)

    def get_lengths(self, formatter, name, max_of_transposed=False):
        """모든 키의 데이터 길이를 계산하는 함수"""
        return {key: self.get_length(key, formatter=formatter, name=name, max_of_transposed=max_of_transposed) for key in self.keys}

    def sorted_by_len(self, reverse=False, **kwargs):
        """길이순으로 정렬된 데이터셋을 반환하는 함수"""
        new_keys = [key for _, key in sorted([(v, k) for k, v in self.get_lengths(**kwargs).items()], reverse=reverse)]
        return self.change_keys(new_keys)

    def filter_by_len(self, min_len=0, max_len=float('inf'), **kwargs):
        """길이로 필터링된 데이터셋을 반환하는 함수"""
        new_keys = [k for k, v in self.get_lengths(**kwargs).items() if min_len<=v<=max_len]
        return self.change_keys(new_keys)

    def cut_to_query_count(self, max_count, from_end=False):
        """
        쿼리 개수를 제한하는 함수
        
        Args:
            max_count: 최대 쿼리 개수
            from_end: 끝에서부터 자를지 여부
        
        Returns:
            쿼리 개수가 제한된 새로운 ArcDataset
        """
        new_queries = {}
        for k in self.keys:
            new_queries[k] = q = self.queries[k]
            while len(q['train'])>max_count: 
                q['train'] = q['train'][:-1] if from_end else q['train'][1:]
        return self.__class__(queries=new_queries, replies=self.replies, keys=[k for k in self.keys])

    def cut_to_len(self, formatter, name, max_len, max_new_tokens='auto', from_end=False, quiet=False, **kwargs):
        """
        최대 길이에 맞춰 데이터를 자르는 함수
        
        Args:
            formatter: 포맷터 객체
            name: 계산할 부분
            max_len: 최대 길이
            max_new_tokens: 최대 새 토큰 수
            from_end: 끝에서부터 자를지 여부
            quiet: 조용한 모드 여부
        
        Returns:
            길이가 조정된 새로운 ArcDataset
        """
        if max_new_tokens:
            if max_new_tokens=='auto': max_new_tokens = formatter.max_new_tokens()
            max_len_old, max_len = max_len, max_len - max_new_tokens
            if not quiet: print(f'*** 작업 크기를 최대 {max_len_old} 토큰 ({max_len} 입력 + {max_new_tokens} 생성)으로 축소 중...')
        elif not quiet: print(f'*** 작업 크기를 최대 {max_len} 토큰으로 축소 중...')
        
        temp_ds = self.change_keys(self.keys)
        new_keys = []
        new_queries = {}
        new_replies = {}
        
        for key in (self.keys if quiet else tqdm(self.keys, file=sys.stdout)):
            reply = temp_ds.replies.get(key)
            # 길이가 초과하는 동안 예제를 제거
            while max_len<temp_ds.get_length(key, formatter=formatter, name=name, **kwargs):
                query = temp_ds.queries[key]
                if not key.split('.')[-1].startswith('ex'): 
                    key = f"{key}.ex{''.join(map(str, range(len(query['train']))))}"
                key_split = key.split('.')
                assert key_split[-1].startswith('ex')
                key = '.'.join(key_split[:-1] + [f'ex{key_split[-1][2:-1] if from_end else key_split[-1][3:]}'])
                temp_ds.queries[key] = {k: ((v[:-1] if from_end else v[1:]) if k=='train' else v) for k, v in query.items()}
                if reply is not None: temp_ds.replies[key] = reply
            new_keys.append(key)
            new_queries[key] = temp_ds.queries[key]
            if reply is not None: new_replies[key] = reply
        return self.__class__(keys=new_keys, queries=new_queries, replies=new_replies)

    def shuffle_ex(self, perm=None, keep_max=None):
        """
        예제 순서를 섞는 함수
        
        Args:
            perm: 사용할 순열 (None이면 무작위)
            keep_max: 유지할 최대 예제 수
        
        Returns:
            예제가 섞인 새로운 ArcDataset
        """
        new_keys = []
        new_queries = {}
        new_replies = {}
        for key in self.keys:
            n = len(self.queries[key]['train'])
            p = np.random.permutation(n) if perm is None else perm
            if keep_max is not None: p = p[:keep_max]
            # 키에 예제 순서 정보 추가
            new_key = f'{key}.ex' + ('-' if (p.max()>9) else '').join(map(str, p.tolist()))
            new_keys.append(new_key)
            new_queries[new_key] = {k: (np.array(v, dtype=object)[p].tolist() if k=='train' else v) for k, v in self.queries[key].items()}
            if key in self.replies: new_replies[new_key] = self.replies[key]
        return self.__class__(queries=new_queries, replies=new_replies, keys=new_keys)

    def shuffle_rp(self, keep_max=None):
        """
        테스트 케이스 순서를 섞는 함수
        
        Args:
            keep_max: 유지할 최대 테스트 케이스 수
        
        Returns:
            테스트 케이스가 섞인 새로운 ArcDataset
        """
        new_keys = []
        new_queries = {}
        new_replies = {}
        for key in self.keys:
            n = len(self.queries[key]['test'])
            p = np.random.permutation(n)
            if keep_max is not None: p = p[:keep_max]
            # 키에 테스트 케이스 순서 정보 추가
            new_key = f'{key}.rp' + ('-' if (p.max()>9) else '').join(map(str, p.tolist()))
            new_keys.append(new_key)
            new_queries[new_key] = {k: (np.array(v, dtype=object)[p].tolist() if k=='test' else v) for k, v in self.queries[key].items()}
            if key in self.replies: new_replies[new_key] = np.array(self.replies[key], dtype=object)[p].tolist()
        return self.__class__(queries=new_queries, replies=new_replies, keys=new_keys)

    def append_to_keys(self, text):
        """
        모든 키에 텍스트를 추가하는 함수
        
        Args:
            text: 추가할 텍스트
        
        Returns:
            키가 수정된 새로운 ArcDataset
        """
        return self.change_keys([f'{k}{text}' for k in self.keys])

    def random_select(self, n):
        """
        n개 그룹 중에서 무작위로 하나씩 선택하는 함수
        
        Args:
            n: 그룹 수
        
        Returns:
            무작위 선택된 새로운 ArcDataset
        """
        keys = np.array(self.keys).reshape(n, -1).T
        choice = np.random.randint(0, n, size=[len(keys)])
        return self.change_keys(keys[np.arange(len(keys)), choice])

    def augment(self, tp=False, rot=False, n=1, perm=None, perm_append=False, shfl_keys=False, shfl_ex=False, seed=None, quiet=False, inputs_only=False):
        """
        데이터 증강을 수행하는 메인 함수
        
        Args:
            tp: 전치 변환 사용 여부 ('rand'면 무작위 선택)
            rot: 회전 변환 사용 여부 ('rand'면 무작위 선택)
            n: 변환 횟수
            perm: 순열 타입 (None, 'rnd_col', 'rnd_all', 'cnt_col', 'cnt_all')
            perm_append: 순열 변환을 추가로 유지할지 여부
            shfl_keys: 키 셔플 여부
            shfl_ex: 예제 셔플 여부
            seed: 랜덤 시드
            quiet: 조용한 모드 여부
            inputs_only: 입력만 변환할지 여부
        
        Returns:
            증강된 새로운 ArcDataset
        """
        if not quiet: print(f"*** 데이터셋 증강{' (입력만)' if inputs_only else ''} 중...")
        np.random.seed(seed)
        d = self
        
        # 전치 변환
        if tp: d = d.mod(np.transpose, keep=True, inputs_only=inputs_only)
        if tp=='rand': d = d.random_select(n=2)
        
        # 회전 변환
        if rot: d = d.mod(np.rot90, n=3, keep=True, inputs_only=inputs_only)
        if rot=='rand': d = d.random_select(n=4)
        
        # 순열 변환
        if perm is None and n<=1: d = d.shuffled() if shfl_keys else d
        else: d = d.mod(*([np.copy] if perm is None else globals()[f"permute_{perm}"]), n=n, shuffle=shfl_keys, keep=perm_append, inputs_only=inputs_only)
        
        # 예제 셔플
        np.random.seed(seed)
        if shfl_ex: d = d.shuffle_ex()
        return d

    def remove_replies(self):
        """답안을 제거한 새로운 데이터셋을 반환하는 함수"""
        return self.__class__(queries=self.queries, replies={}, keys=[k for k in self.keys])

    def split_at_pos(self, pos, random_seed=None):
        """
        지정된 위치에서 데이터셋을 분할하는 함수
        
        Args:
            pos: 분할 위치 (정수 또는 비율)
            random_seed: 랜덤 시드 (섞기용)
        
        Returns:
            분할된 두 개의 ArcDataset 튜플
        """
        keys = self.keys
        if random_seed is not None:
            np.random.seed(random_seed)
            keys = np.random.permutation(keys)
        if isinstance(pos, float): pos = int(pos * len(self.keys) + 0.5)
        keys_split = [keys[:pos], keys[pos:]]
        return tuple(self.change_keys(new_keys, keep_flags=True) for new_keys in keys_split)

    def get_submission(self, results=None):
        """
        제출용 형식의 결과를 생성하는 함수
        
        Args:
            results: 결과 딕셔너리 (선택사항)
        
        Returns:
            제출용 형식의 딕셔너리
        """
        assert self.is_orig==True, '원본 데이터셋에서만 실행해야 합니다.'
        # 각 문제마다 2번의 시도 기회를 가진 제출 형식 생성
        submission = {k: [{f'attempt_{i+1}': [[0]] for i in range(2)} for _ in range(len(self.queries[k]['test']))] for k in self.keys}
        if results is not None: self.fill_submission(results, submission)
        return submission

    @staticmethod
    def fill_submission(results, submission):
        """
        결과를 제출 형식에 채우는 정적 메서드
        
        Args:
            results: 결과 딕셔너리
            submission: 제출 형식 딕셔너리
        """
        print(f'*** {len(results)}개 출력에 대한 제출 생성 중...')
        for k, v in results.items():
            base_id, base_nr = k.split('_')
            target_dict = submission[base_id][int(base_nr)]
            for i, g in enumerate(v[:len(target_dict)]):
                target_dict[f'attempt_{i+1}'] = g.tolist()

    def validate_submission(self, submission):
        """
        제출 결과를 검증하는 함수
        
        Args:
            submission: 제출 딕셔너리
        
        Returns:
            점수 (0~1 사이의 값)
        """
        assert self.is_orig==True, '원본 데이터셋에서만 실행해야 합니다.'
        score = 0
        for k, v in self.replies.items():
            for i, r in enumerate(v):
                # 두 번의 시도 중 하나라도 맞으면 점수 획득
                for attempt in ['attempt_1', 'attempt_2']:
                    if np.array_equal(r, submission[k][i][attempt]):
                        score += 1 / len(v)
                        break
        return score

def get_class_MyDataCollator(cache=[]):
    """
    커스텀 데이터 콜레이터 클래스를 반환하는 함수 (싱글톤 패턴)
    
    Args:
        cache: 캐시 리스트 (싱글톤 구현용)
    
    Returns:
        MyDataCollator 클래스
    """
    if not cache:
        from trl import DataCollatorForCompletionOnlyLM
        
        class MyDataCollator(DataCollatorForCompletionOnlyLM):
            """ARC 작업에 특화된 커스텀 데이터 콜레이터"""
            
            def setup(self, out2_token_id=None, fault_token_id=None, fault_freq=0, sample_tries=8, mask_first_output=False):
                """
                데이터 콜레이터 설정
                
                Args:
                    out2_token_id: 두 번째 출력 토큰 ID
                    fault_token_id: 오류 토큰 ID
                    fault_freq: 오류 주입 빈도
                    sample_tries: 샘플링 시도 횟수
                    mask_first_output: 첫 번째 출력 마스킹 여부
                
                Returns:
                    설정된 self 객체
                """
                self.out2_token_id = out2_token_id
                self.fault_token_id = fault_token_id
                self.fault_freq = fault_freq
                self.sample_tries = sample_tries
                self.mask_first_output = mask_first_output
                return self

            def torch_call(self, examples):
                """
                배치 처리 메인 함수
                
                Args:
                    examples: 예제 리스트
                
                Returns:
                    처리된 배치
                """
                batch = super().torch_call(examples)
                
                # 두 번째 출력 토큰 처리
                if self.out2_token_id is not None:
                    assert not self.fault_freq
                    for i in range(len(batch['input_ids'])):
                        end_pos = ((batch['labels'][i] != -100              ).nonzero().max()).item() + 1
                        mid_pos = ((batch['labels'][i] == self.out2_token_id).nonzero().max()).item() + 1
                        beg_pos = mid_pos - (end_pos - mid_pos)
                        # 첫 번째 출력을 두 번째 출력으로 복사
                        batch['labels'][i][beg_pos:mid_pos] = batch['labels'][i][mid_pos:end_pos]
                
                # 오류 주입 처리
                elif self.fault_freq:
                    for i in range(len(batch['input_ids'])):
                        end_pos = ((batch['labels'][i] != -100).nonzero().max()).item() + 1
                        
                        # 동적 오류 빈도 계산
                        if not isinstance(self.fault_freq, float):
                            eos_token_id = batch['labels'][i][end_pos - 1]
                            num_examples = (batch['labels'][i] == eos_token_id).sum().item() - 1
                            fault_freq = self.fault_freq[num_examples]
                        else: 
                            fault_freq = self.fault_freq
                        
                        # 확률적 오류 주입
                        if random.random() < fault_freq:
                            beg_pos = ((batch['labels'][i][:end_pos]==-100).nonzero().max()).item() + 1
                            fault_pos = random.randint(beg_pos, end_pos-2)
                            fault_tok = batch['labels'][i][fault_pos].item()
                            
                            # 다른 토큰으로 교체 시도
                            for t in range(self.sample_tries):
                                new_tok = batch['labels'][i][random.randint(beg_pos, end_pos-2)].item()
                                if fault_tok!=new_tok:
                                    batch['input_ids'][i][fault_pos] = new_tok
                                    # 오류 후 모든 토큰을 오류 토큰으로 마스킹
                                    batch['labels'][i][fault_pos+1:end_pos] = self.fault_token_id
                                    break
                
                # 첫 번째 출력 마스킹
                for i in range(len(batch['labels'])):
                    for _ in range(self.mask_first_output):
                        beg_pos = ((batch['labels'][i] != -100).nonzero().min()).item()
                        mid_pos = ((batch['labels'][i][beg_pos:] == -100).nonzero().min()).item() + beg_pos
                        end_pos = ((batch['labels'][i] != -100).nonzero().max()).item() + 1
                        if mid_pos<end_pos: batch['labels'][i][beg_pos:mid_pos] = -100
                return batch
        cache.append(MyDataCollator)
    return cache[0]

class ArcFormatter(object):
    """ARC 데이터를 텍스트 형식으로 포맷팅하는 클래스"""
    
    def __init__(self, inp_prefix, out_prefix, arr_sep, out2_use=False, out2_token=None, arr_beg='', arr_end='', pretext='', pre_out=None, exa_sep='', exa_end='', qry_prefix=None, rpl_prefix=None, rpl_sep=None, dec_sep=None, min_wid=0, min_pad='', pretext_corpus_split='', masking=0, tokenizer=None, collator_kwargs={}, repeat_input_aug=None, repeat_input_pre=None):
        """
        ArcFormatter 초기화
        
        Args:
            inp_prefix: 입력 접두사
            out_prefix: 출력 접두사
            arr_sep: 배열 분리자
            out2_use: 두 번째 출력 사용 여부
            out2_token: 두 번째 출력 토큰
            arr_beg: 배열 시작 문자
            arr_end: 배열 끝 문자
            pretext: 전문(前文)
            pre_out: 출력 전 텍스트
            exa_sep: 예제 분리자
            exa_end: 예제 끝 문자
            qry_prefix: 쿼리 접두사
            rpl_prefix: 응답 접두사
            rpl_sep: 응답 분리자
            dec_sep: 디코딩 분리자
            min_wid: 최소 너비
            min_pad: 최소 패딩 문자
            pretext_corpus_split: 전문 코퍼스 분할 문자
            masking: 마스킹 모드
            tokenizer: 토크나이저
            collator_kwargs: 콜레이터 인자들
            repeat_input_aug: 입력 반복 증강 함수
            repeat_input_pre: 입력 반복 접두사
        """
        self.tokenizer = tokenizer
        self.inp_prefix = inp_prefix
        self.out_prefix = out_prefix
        self.out2_token = out2_token
        self.out2_use = out2_use
        assert not out2_use or out2_token is not None
        assert not out2_use or masking in [1, 2]
        assert masking!=2 or out2_use or rpl_prefix is not None
        
        # 기본값 설정
        self.qry_prefix = qry_prefix if qry_prefix is not None else inp_prefix
        self.rpl_prefix = rpl_prefix if rpl_prefix is not None else out_prefix
        self.rpl_sep = rpl_sep if rpl_sep is not None else self.rpl_prefix
        self.arr_sep = arr_sep
        self.arr_beg = arr_beg
        self.arr_end = arr_end
        self.pretext = pretext
        self.pre_out = pre_out
        self.pre_out_empty = ['']*99  # 빈 출력 전 텍스트
        self.pretext_corpus_split = pretext_corpus_split
        self.exa_sep = exa_sep
        self.exa_end = exa_end
        self.dec_sep = arr_sep if dec_sep is None else dec_sep
        self.min_wid = min_wid
        self.min_pad = min_pad
        self.masking = masking
        self.collator_kwargs = collator_kwargs
        self.repeat_input_aug = repeat_input_aug
        self.repeat_input_pre = repeat_input_pre

    def fmt_array(self, array):
        """
        2D 배열을 텍스트 형식으로 포맷팅하는 함수
        
        Args:
            array: 2D 배열
        
        Returns:
            포맷된 텍스트 문자열
        """
        return self.arr_beg + self.arr_sep.join(
            str(row).replace(' ', '').replace(',', '').replace('[', '').replace(']', '') + 
            self.min_pad*max(0, self.min_wid-len(row)) 
            for row in array
        ) + self.arr_end

    def get_pre_out(self, pretext_split):
        """
        출력 전 텍스트를 가져오는 함수
        
        Args:
            pretext_split: 전문 분할 여부
        
        Returns:
            출력 전 텍스트 리스트
        """
        if self.pre_out is None: return self.pre_out_empty
        if pretext_split: return [self.pretext_corpus_split.join(list(p) + ['']) for p in self.pre_out]
        return self.pre_out

    def fmt_train(self, train, last_is_challenge=False, pretext_split=False):
        """
        훈련 예제들을 포맷팅하는 함수
        
        Args:
            train: 훈련 예제 리스트
            last_is_challenge: 마지막이 도전 문제인지 여부
            pretext_split: 전문 분할 여부
        
        Returns:
            포맷된 훈련 데이터 문자열
        """
        po = self.get_pre_out(pretext_split=pretext_split)
        ex = []
        for i, x in enumerate(train):
            if last_is_challenge and i+1==len(train):
                # 마지막이 도전 문제인 경우
                formatted_ex = f"{self.fmt_query([x], i, pretext_split=pretext_split)}{self.fmt_reply([x['output']])}"
            else:
                # 일반 훈련 예제
                formatted_ex = f"{self.inp_prefix}{self.fmt_array(x['input'])}{self.repeat_input(x, no_aug=pretext_split)}{po[i]}{self.out_prefix}{self.fmt_array(x['output'])}"
            ex.append(formatted_ex)
        
        pre = self.pretext_corpus_split.join(list(self.pretext)+['']) if pretext_split else self.pretext
        end = '' if last_is_challenge else (self.exa_end + self.tokenizer.eos_token)
        return pre + (self.exa_end + self.tokenizer.eos_token + self.exa_sep).join(ex) + end

    def fmt_query(self, query, i, pretext_split=False):
        """
        쿼리를 포맷팅하는 함수
        
        Args:
            query: 쿼리 리스트
            i: 인덱스
            pretext_split: 전문 분할 여부
        
        Returns:
            포맷된 쿼리 문자열
        """
        po = self.get_pre_out(pretext_split=pretext_split)
        return ''.join(f"{self.qry_prefix}{self.fmt_array(x['input'])}{self.repeat_input(x, no_aug=pretext_split)}{po[i]}{self.rpl_prefix}" for x in query[:1])

    def repeat_input(self, x, no_aug=False):
        """
        입력 반복 기능
        
        Args:
            x: 입력 데이터
            no_aug: 증강 없음 여부
        
        Returns:
            반복된 입력 문자열
        """
        if self.repeat_input_aug is None: return ''
        return f"{self.repeat_input_pre}{self.fmt_array(((lambda x: x) if no_aug else self.repeat_input_aug)(x['input']))}"

    def fmt_reply(self, reply, fault=None):
        """
        응답을 포맷팅하는 함수
        
        Args:
            reply: 응답 리스트
            fault: 오류 데이터 (선택사항)
        
        Returns:
            포맷된 응답 문자열
        """
        ids = self.fmt_array(reply[0]) + self.exa_end + self.tokenizer.eos_token
        if self.out2_use:
            # 두 번째 출력 사용 시
            if fault is None: fault = reply
            ids = self.fmt_array(fault[0]) + self.exa_end + self.out2_token + ids
        return ids

    def quick_test(self, decoded, done):
        """
        디코딩된 결과에 대한 빠른 테스트
        
        Args:
            decoded: 디코딩된 문자열
            done: 완료 여부
        
        Returns:
            테스트 통과 여부
        """
        sp = decoded.split(self.tokenizer.eos_token)[0].split(self.dec_sep)
        sl = len(sp[0])
        is_prefix = sl>0 and len(sp[-1])<=sl and (len(sp)==1 or len(sp[-2])==sl) and all(x.isdigit() for x in sp[-1])
        return is_prefix and (not done or len(sp[-1])==0 or len(sp[-1])==sl)

    @staticmethod
    def is_valid_solution(guess):
        """
        추측이 유효한 솔루션인지 확인하는 정적 메서드
        
        Args:
            guess: 추측 배열
        
        Returns:
            유효성 여부
        """
        return isinstance(guess, np.ndarray) and guess.ndim == 2 and all(0 < x <= 30 for x in guess.shape)

    def max_new_tokens(self, safety_margin=1):
        """
        최대 새 토큰 수를 계산하는 함수
        
        Args:
            safety_margin: 안전 마진
        
        Returns:
            최대 새 토큰 수
        """
        # 최대 크기 응답 (30x30)으로 계산
        max_sized_reply = np.zeros([30, 30], dtype=int)
        tokenized = self.tokenizer(self.fmt_reply([max_sized_reply]))['input_ids']
        max_new_tokens = len(tokenized)
        if tokenized[0]==self.tokenizer.bos_token_id: max_new_tokens -= 1
        return max_new_tokens + safety_margin

    def de_tokenize(self, tokens, scores=None):
        """
        토큰을 디토크나이즈하는 함수
        
        Args:
            tokens: 토큰 배열
            scores: 점수 배열 (선택사항)
        
        Returns:
            (출력 길이, 점수 값, 디토크나이즈된 텍스트, 점수들) 튜플
        """
        import torch
        tokens_cut = cut_at_token(tokens, self.tokenizer.eos_token_id)
        de_tokenized = self.tokenizer.batch_decode([tokens_cut])[0]
        score_val = None
        
        if scores is not None:
            tokens_with_eos = tokens[:len(tokens_cut)+1]
            # 로그 소프트맥스로 점수 값 계산
            score_val = torch.nn.functional.log_softmax(torch.tensor(scores), dim=-1).numpy().copy()[np.arange(len(tokens_with_eos)), tokens_with_eos].sum()
            
            # 숫자 토큰들만 추출
            number_token_ids = [self.tokenizer.vocab[k] for k in map(str, range(10))]
            fault_token_id = self.collator_kwargs.get('fault_token_id')
            if fault_token_id is not None: number_token_ids.append(fault_token_id)
            number_token_ids = np.array(number_token_ids)
            number_positions = (tokens_cut[..., np.newaxis] == number_token_ids).any(-1)
            scores = scores[:len(tokens_cut), number_token_ids][number_positions]
            scores = torch.nn.functional.log_softmax(torch.tensor(scores), dim=-1)[:, :10].numpy().copy()
        
        return max(len(tokens)+1, len(tokens_cut)), score_val, de_tokenized, scores

    def decode_to_array_single(self, text, score=None, limit_rows=30):
        """
        단일 텍스트를 배열로 디코딩하는 함수
        
        Args:
            text: 디코딩할 텍스트
            score: 점수 배열 (선택사항)
            limit_rows: 최대 행 수 제한
        
        Returns:
            디코딩 결과 딕셔너리
        """
        try:
            # 텍스트를 행별로 분할하고 숫자만 추출
            by_rows = [row for row in [[int(x) for x in line if x.isdigit()] for line in text.split(self.dec_sep)] if len(row)]
            if limit_rows and len(by_rows) > limit_rows:
                by_rows = by_rows[:limit_rows]
                limited = True
            else: 
                limited = False
            
            decoded = np.array(by_rows, dtype=int)
            if self.is_valid_solution(decoded):
                try:
                    assert score is not None
                    decoded_flat = decoded.ravel()
                    if limited: score = score[:len(decoded_flat)]
                    
                    # 다양한 점수 형태 계산
                    score_all = score.reshape(decoded.shape + score.shape[1:])
                    score_result = score[range(len(decoded_flat)), decoded_flat]
                    score_reshaped = score_result.reshape(decoded.shape)
                    score_cum_reshaped = score_result.cumsum().reshape(score_reshaped.shape)
                    score_all_cum = score_cum_reshaped[..., np.newaxis] - score_reshaped[..., np.newaxis] + score_all
                except: 
                    # 점수 계산 실패 시 무한대 값으로 채움
                    score_reshaped = score_cum_reshaped = np.full(decoded.shape, -float('inf'))
                
                return {
                    'output': decoded, 
                    'score': score_reshaped, 
                    'score_cum': score_cum_reshaped, 
                    'score_all': score_all, 
                    'score_all_cum': score_all_cum
                }
        except: 
            pass
        return {}

    def decode_to_array(self, text, score=None, limit_rows=30):
        """
        텍스트를 배열로 디코딩하는 메인 함수
        
        Args:
            text: 디코딩할 텍스트
            score: 점수 배열 (선택사항)
            limit_rows: 최대 행 수 제한
        
        Returns:
            디코딩 결과 리스트
        """
        if not self.out2_use: 
            text, score = [text], [score]
        else:
            # 두 번째 출력 토큰으로 분할
            text = text.split(self.out2_token)
            if score is None: 
                score = [None]*len(text)
            else:
                # 텍스트 길이에 따라 점수 분할
                lengths = np.cumsum([len(list(filter(str.isdigit, t))) for t in text])
                score = [score[s:e] for s, e in zip([0]+lengths[:-1].tolist(), lengths)]
        
        return [self.decode_to_array_single(t, s) for t, s in zip(text, score)]

    def get_corpus(self):
        """
        토크나이저 학습용 코퍼스를 생성하는 함수
        
        Returns:
            코퍼스 텍스트
        """
        try:
            old_min_wid, self.min_wid = self.min_wid, min(self.min_wid, 2)
            # 0-9 숫자로 구성된 간단한 예제 생성
            return self.fmt_train([{'input': [[i] for i in range(10)], 'output': [[i] for i in range(10)]}]*3, last_is_challenge=True, pretext_split=True)
        finally: 
            self.min_wid = old_min_wid

    def get_data_collator(self):
        """
        데이터 콜레이터를 생성하는 함수
        
        Returns:
            데이터 콜레이터 객체 또는 None
        """
        if not self.masking: return None
        
        from transformers import DataCollatorForLanguageModeling
        collator_params = dict(tokenizer=self.tokenizer, mlm=False)
        
        # 두 번째 출력 토큰 ID 설정
        pass_out2_token = self.tokenizer.vocab[self.out2_token] if self.out2_use and self.masking==1 else None
        
        if self.masking:
            assert not self.collator_kwargs.get('mask_first_output') or self.masking==1
            # 커스텀 콜레이터 생성
            data_collator = get_class_MyDataCollator()(
                **collator_params,
                instruction_template=[self.inp_prefix, self.tokenizer.bos_token][self.masking - 1],
                response_template=[self.out_prefix, (self.out2_token if self.out2_use else self.rpl_sep)][self.masking - 1],
            ).setup(out2_token_id=pass_out2_token, **self.collator_kwargs)
        else:
            assert not self.collator_kwargs, '마스킹이 켜져있을 때만 지원됩니다'
            data_collator = DataCollatorForLanguageModeling(**collator_params)
        
        return data_collator

    def get_output_token_ids(self):
        """
        출력에 사용되는 토큰 ID들을 반환하는 함수
        
        Returns:
            출력 토큰 ID 리스트
        """
        assert not self.out2_use
        # 숫자 토큰들 (0-9)
        num_tokens = [self.tokenizer.vocab[str(i)] for i in range(10)]
        
        # 분리자 토큰들
        sep_tokens = []
        for txt in [self.arr_beg, self.arr_sep, self.arr_end, self.exa_sep]:
            if txt:
                for tok in self.tokenizer(txt)['input_ids'][1:]:
                    sep_tokens.append(tok)
        sep_tokens.append(self.tokenizer.eos_token_id)
        
        return num_tokens + sorted(set(sep_tokens))

# 사전 정의된 포맷터들
ArcFormatter_pretext2 = lambda **kwargs: ArcFormatter(
    masking=1, 
    inp_prefix='I', 
    out_prefix='O', 
    arr_sep='\n', 
    arr_end='\n', 
    pretext='ABCDEFGHJKLMNPQRSTUVWXYZ',  # I와 O를 제외한 알파벳
    pretext_corpus_split='\n', 
    **kwargs
)

ArcFormatter_pretext3 = lambda **kwargs: ArcFormatter(
    masking=1, 
    inp_prefix='I', 
    out_prefix='O', 
    arr_sep='\n', 
    arr_end='\n', 
    pretext='ABCDEFGHJKLMNPQRSTUVWXYZabcdefghjklmnpqrstuvwxyz',  # 대소문자 알파벳 (I, O, i, o 제외)
    pretext_corpus_split='\n', 
    **kwargs
)

ArcFormatter_premix_2 = lambda **kwargs: ArcFormatter(
    masking=1, 
    inp_prefix='I', 
    out_prefix='O', 
    arr_sep='\n', 
    arr_end='\n', 
    pretext='ABCDEFGHJKLMNPQRSTUVWXYZ', 
    pre_out=['+/-=']*99,  # 출력 전에 수학 기호 추가
    pretext_corpus_split='\n', 
    **kwargs
)

ArcFormatter_premix_3 = lambda **kwargs: ArcFormatter(
    masking=1, 
    inp_prefix='I', 
    out_prefix='O', 
    arr_sep='\n', 
    arr_end='\n', 
    pretext='ABCDEFGHJKLMNPQRSTUVWXYZabcdefghjklmnpqrstuvwxyz', 
    pre_out=['+/-=']*99,  # 출력 전에 수학 기호 추가
    pretext_corpus_split='\n', 
    **kwargs
)

# 사용 가능한 포맷터들의 딕셔너리
available_formatters = dict(
    ArcFormatter_pretext2=ArcFormatter_pretext2,
    ArcFormatter_pretext3=ArcFormatter_pretext3,
    ArcFormatter_premix_2=ArcFormatter_premix_2,
    ArcFormatter_premix_3=ArcFormatter_premix_3,
)

In [ ]:
%%writefile selection.py
import numpy as np

def hashable(guess):
    """
    2D 배열을 해시 가능한 튜플로 변환하는 함수
    
    Args:
        guess: 2D numpy 배열 형태의 추측
    
    Returns:
        중첩 튜플 형태의 해시 가능한 객체
    """
    return tuple(map(tuple, guess))

def make_unique(guess_list, indices=None):
    """
    추측 리스트에서 중복을 제거하는 함수
    
    Args:
        guess_list: 추측들의 리스트
        indices: 인덱스 리스트 (선택사항)
    
    Returns:
        중복이 제거된 추측 리스트 (또는 추측과 인덱스 튜플)
    """
    used = set()  # 이미 사용된 해시값들을 저장
    out = []      # 고유한 추측들
    out_ind = []  # 고유한 추측들의 인덱스
    
    for i, g in enumerate(guess_list):
        h = hashable(g)
        if h not in used:
            used.add(h)
            out.append(np.array(g))
            if indices is not None: 
                out_ind.append(indices[i])
    
    return out if indices is None else (out, out_ind)

def first_only(guesses):
    """
    첫 번째 추측만 반환하는 선택 알고리즘
    
    Args:
        guesses: 추측 딕셔너리 {key: {'output': array, ...}}
    
    Returns:
        첫 번째 추측만 포함된 리스트
    """
    return [g['output'] for g in guesses.values()][:1]

def keep_order(guesses):
    """
    모든 추측을 원래 순서대로 유지하는 선택 알고리즘
    
    Args:
        guesses: 추측 딕셔너리
    
    Returns:
        모든 추측의 출력 배열 리스트
    """
    return [g['output'] for g in guesses.values()]

def keep_order_unique(guesses):
    """
    원래 순서를 유지하면서 중복을 제거하는 선택 알고리즘
    
    Args:
        guesses: 추측 딕셔너리
    
    Returns:
        중복이 제거된 고유한 추측들의 리스트
    """
    return make_unique(keep_order(guesses))

def get_best_shape_by_score(guess_list, getter, once_per_result=True):
    """
    점수 기반으로 최고의 형태(shape)를 찾는 함수
    
    Args:
        guess_list: 추측 리스트
        getter: 점수를 계산하는 함수
        once_per_result: 동일한 결과당 한 번만 계산할지 여부
    
    Returns:
        (최고 점수, 형태, 인덱스들) 튜플
    """
    seen_outputs = set()  # 이미 본 출력들
    shape_scores = {}     # 형태별 점수와 인덱스 저장
    
    for i, g in enumerate(guess_list):
        shape = tuple(g['output'].shape)  # 배열의 형태 (높이, 너비)
        scores = shape_scores[shape] = shape_scores.get(shape, [[], []])
        scores[1].append(i)  # 인덱스 추가
        
        h = hashable(g['output'])
        if h in seen_outputs: continue
        if once_per_result: seen_outputs.add(h)
        scores[0].append(g)  # 추측 추가
    
    # 각 형태별로 점수 계산 후 정렬
    shape_scores = [(getter(scores), shape, indices) for shape, (scores, indices) in shape_scores.items()]
    shape_scores = sorted(shape_scores, key=(lambda x: x[0]), reverse=True)
    return shape_scores[0]  # 최고 점수의 형태 반환

def score_sum(guesses, getter, shape_getter=None, prefer_common_shape=True):
    """
    점수 합계를 기반으로 추측을 정렬하는 일반적인 함수
    
    Args:
        guesses: 추측 딕셔너리
        getter: 점수를 계산하는 함수
        shape_getter: 형태 점수를 계산하는 함수 (기본값: getter와 동일)
        prefer_common_shape: 일반적인 형태를 선호할지 여부
    
    Returns:
        점수순으로 정렬된 출력 배열들의 리스트
    """
    if shape_getter is None: shape_getter = getter
    guess_list = list(guesses.values())
    
    # 일반적인 형태를 선호하는 경우, 해당 인덱스들을 찾음
    common_shape_indices = set(get_best_shape_by_score(guess_list, shape_getter)[2]) if prefer_common_shape else []
    
    scores = {}
    for i, g in enumerate(guess_list):
        h = hashable(g['output'])
        # [일반적인_형태_여부, 추측들, 출력_배열]
        x = scores[h] = scores.get(h, [i in common_shape_indices, [], g['output']])
        x[1].append(g)
    
    # 점수 계산 및 정렬: (일반적인_형태_여부, 계산된_점수, 출력_배열)
    scores = [(cs, getter(sc), o) for cs, sc, o in scores.values()]
    scores = sorted(scores, key=(lambda x: x[:2]), reverse=True)
    ordered_outputs = [x[-1] for x in scores]
    return ordered_outputs

# 확률 합계를 계산하는 getter 함수
getter_all_probsum = lambda guesses: sum(np.exp(g['score_val']) for g in guesses)

def score_all_probsum(guesses): 
    """
    모든 확률의 합계를 기반으로 추측을 선택하는 알고리즘
    
    Args:
        guesses: 추측 딕셔너리
    
    Returns:
        확률 합계순으로 정렬된 추측들
    """
    return score_sum(guesses, getter_all_probsum)

def getter_full_probmul(p):
    """
    전체 확률 곱셈을 위한 getter 생성 함수
    
    Args:
        p: 기준선(baseline) 값
    
    Returns:
        확률 곱셈을 계산하는 getter 함수
    """
    def _getter(guesses, baseline=p):
        """
        추론 점수와 증강 점수를 결합하여 전체 점수를 계산
        
        Args:
            guesses: 추측 리스트
            baseline: 기준선 값
        
        Returns:
            결합된 점수
        """
        # 추론 점수: 각 추측의 점수에 기준선을 더한 합
        inf_score = sum([g['score_val']+baseline for g in guesses])
        
        # 증강 점수: 다중 점수들의 평균 (기준선 포함)
        aug_score = np.mean([sum(s+baseline for s in g['score_multi_nl']) for g in guesses])
        
        return inf_score + aug_score
    return _getter

def score_full_probmul_3(guesses): 
    """
    기준선 3을 사용한 전체 확률 곱셈 기반 선택 알고리즘
    
    Args:
        guesses: 추측 딕셔너리
    
    Returns:
        전체 확률 곱셈 점수순으로 정렬된 추측들
    """
    return score_sum(guesses, getter_full_probmul(3), prefer_common_shape=False)

# 사용 가능한 선택 알고리즘들의 리스트
selection_algorithms = [
    first_only,            # 첫 번째만 선택
    keep_order,            # 순서 유지
    keep_order_unique,     # 순서 유지 + 중복 제거
    score_all_probsum,     # 확률 합계 기반
    score_full_probmul_3,  # 전체 확률 곱셈 기반 (기준선=3)
]

In [ ]:
%%writefile async_tools.py
import sys
import asyncio

async def stream_reader(stream, id, to):
    """
    스트림에서 데이터를 비동기적으로 읽고 실시간으로 출력하는 함수
    
    Args:
        stream: 읽을 스트림 (stdout 또는 stderr)
        id: 프로세스 식별자 (None이면 ID 표시 안함)
        to: 출력할 대상 스트림 (None이면 출력 안함)
    """
    # ID 접두사 설정 (None이면 빈 문자열)
    id = '' if id is None else f'{id}. '
    data = b''  # 아직 완성되지 않은 라인의 버퍼
    
    while True:
        # 스트림에서 최대 4096바이트씩 읽기
        read = await stream.read(n=4096)
        if not read: break  # 더 이상 읽을 데이터가 없으면 종료
        
        if to is not None:
            # 개행 문자로 라인 분할 (마지막에 'X' 추가하여 빈 라인 구분)
            *complete_lines, data = (data + read + b'X').splitlines()
            data = data[:-1]  # 'X' 제거
            
            # 완성된 라인들을 출력
            for line in complete_lines:
                line = line.rstrip()  # 끝의 공백 문자 제거
                if line:  # 빈 라인이 아닌 경우에만 출력
                    print(f"{id}{line.decode('utf-8')}", file=to, end='\n', flush=True)

async def wait_for_subprocess(subprocess, print_output=False, id=None):
    """
    단일 서브프로세스의 완료를 기다리면서 출력을 스트리밍하는 함수
    
    Args:
        subprocess: 기다릴 서브프로세스 객체
        print_output: 출력을 콘솔에 표시할지 여부
        id: 프로세스 식별자 (다중 프로세스 실행 시 구분용)
    
    Returns:
        서브프로세스의 종료 코드
    """
    # stdout과 stderr를 동시에 비동기적으로 처리
    await asyncio.gather(
        stream_reader(
            subprocess.stdout, 
            id, 
            (sys.stdout if print_output else None)  # 출력 표시 여부에 따라 대상 설정
        ),
        stream_reader(
            subprocess.stderr, 
            id, 
            (sys.stderr if print_output else None)   # 에러 출력도 동일하게 처리
        ),
    )
    
    # 서브프로세스의 종료를 기다리고 종료 코드 반환
    return await subprocess.wait()

async def wait_for_subprocesses(*processes, print_output=False):
    """
    여러 서브프로세스들의 완료를 동시에 기다리는 함수
    
    Args:
        *processes: 기다릴 서브프로세스들 (가변 인수)
        print_output: 출력을 콘솔에 표시할지 여부
    
    Returns:
        모든 서브프로세스들의 종료 코드 리스트
    """
    # 각 프로세스에 대해 wait_for_subprocess를 비동기적으로 실행
    # 프로세스가 여러 개인 경우에만 ID 부여 (구분용)
    return await asyncio.gather(*[
        wait_for_subprocess(
            p, 
            print_output=print_output, 
            id=i if len(processes) > 1 else None  # 단일 프로세스면 ID 없음
        ) 
        for i, p in enumerate(processes)
    ])

In [ ]:
%%writefile common_stuff.py
# ARC 훈련 및 평가를 위한 공통 설정
from arc_loader import *
from model_runner import *
from selection import *
from async_tools import *
import time

# ===== 파일 경로 설정 =====
tmp_dir = '/kaggle/temp'  # 임시 파일 저장 디렉토리
arc_challenge_file = '/kaggle/input/arc-prize-2025/arc-agi_test_challenges.json'  # ARC 테스트 문제 파일
arc_solutions_file = '/kaggle/input/arc-prize-2025/arc-agi_training_solutions.json'  # ARC 훈련 솔루션 파일
model_temp_storage = os.path.join(tmp_dir, 'finetuned_model')  # 파인튜닝된 모델 저장 경로
infer_temp_storage = os.path.join(tmp_dir, 'inference_outputs')  # 추론 결과 저장 경로
score_temp_storage = os.path.join(tmp_dir, 'inference_scoring')  # 점수 계산 결과 저장 경로

# ===== 데이터셋 로드 =====
arc_test_set = ArcDataset.from_file(arc_challenge_file)  # ARC 테스트 세트 로드
if arc_test_set.is_fake: arc_test_set.load_replies(arc_solutions_file)  # 가짜 테스트 세트인 경우 솔루션 로드
#arc_test_set.is_fake = False  # 전체 실행 강제 (주석 처리됨)
#arc_train_set = ArcDataset.from_file('/kaggle/input/arc-prize-2025/arc-agi_training_challenges.json')  # 훈련 세트 (사용 안함)

# ===== 모델 설정 =====
base_model = '/kaggle/input/wb55l_nemomini_fulleval/transformers/default/1'  # 기본 모델 경로
MyFormatter = ArcFormatter_premix_3  # 사용할 포맷터 (수학 기호 포함 버전)
perm_aug = 'rnd_all'  # 순열 증강 타입 (모든 색상 무작위 순열)
max_seq_length_train = 4224  # 훈련 시 최대 시퀀스 길이
mask_first = 0  # 첫 번째 출력 마스킹 설정

# ===== 훈련 및 추론 설정 =====
train_epochs = 4  # 훈련 에포크 수
multi_gpu_train = True  # 다중 GPU 훈련 사용 여부
multi_gpu_random_split = True  # 다중 GPU 시 무작위 분할 여부
max_seq_length_infer = 8192  # 추론 시 최대 시퀀스 길이
prime_on_single_task = False  # 단일 작업 프라이밍 여부
infer_params = dict(
    min_prob=0.17,  # 터보 DFS 최소 확률 임계값
    store=infer_temp_storage,  # 추론 결과 저장 경로
    use_turbo=True  # 터보 모드 사용 여부
)

# ===== 점수 계산 설정 =====
use_aug_score = True  # 증강 점수 사용 여부
aug_score_params = dict(
    tp=True,  # 전치 변환 사용
    rot=True,  # 회전 변환 사용
    perm=perm_aug,  # 순열 증강 타입
    shfl_ex=True,  # 예제 셔플 사용
    make_unique=True,  # 고유성 확보
    max_len=max_seq_length_infer  # 최대 길이
)
# 제출용 선택 알고리즘 (증강 점수 사용 여부에 따라 결정)
submission_select_algo = score_full_probmul_3 if use_aug_score else score_all_probsum

def prepare_run(model_path, load_lora=None, train=False, gpu=None, **kwargs):
    """
    모델 실행을 위한 준비 함수
    
    Args:
        model_path: 모델 경로
        load_lora: 로드할 LoRA 경로
        train: 훈련 모드 여부
        gpu: 사용할 GPU 번호
        **kwargs: 추가 인자들
    
    Returns:
        (모델, 포맷터) 튜플
    """
    # GPU 설정
    if gpu is not None:
        os.environ["CUDA_DEVICE_ORDER"   ] = "PCI_BUS_ID"
        os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu)

    # 모델, 토크나이저, 포맷터 준비
    model, tokenizer, formatter = prepare_model(
        model=model_path,
        local_files_only=True,  # 로컬 파일만 사용
        mode='unsloth_4bit',  # Unsloth 4비트 모드
        #shrink_embedding=8000,  # 임베딩 축소 (주석 처리됨)
        max_seq_length=max_seq_length_train,
        formatter=MyFormatter,
        # LoRA 설정 (훈련 또는 LoRA 로드 시에만)
        peft=([dict(
            r=64,  # LoRA 랭크 (8, 16, 32, 64, 128 중 선택)
            target_modules=['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj', 'embed_tokens', 'lm_head'],
            lora_alpha=16,  # LoRA 알파 값
            lora_dropout=0,  # LoRA 드롭아웃 (0이 최적화됨)
            bias="none",  # 바이어스 설정 ("none"이 최적화됨)
            use_gradient_checkpointing=True,  # 그래디언트 체크포인팅 사용
            random_state=42,  # 랜덤 시드
            use_rslora=True,  # 랭크 안정화 LoRA 사용
            loftq_config=None,  # LoftQ 설정
        )] if train or load_lora else []) + ([load_lora] if load_lora else []),
        **kwargs
    )
    
    # 훈련 시 첫 번째 출력 마스킹 설정
    if train and mask_first: 
        formatter.collator_kwargs.update(mask_first_output=mask_first)

    return model, formatter

def prepare_dataset(formatter, train, gpu=None):
    """
    데이터셋을 준비하는 함수
    
    Args:
        formatter: 포맷터 객체
        train: 훈련 모드 여부
        gpu: 사용할 GPU 번호
    
    Returns:
        준비된 ArcDataset
    """
    ds = arc_test_set
    
    # 다중 GPU 훈련 시 데이터 분할
    if multi_gpu_train and gpu is not None:
        if multi_gpu_random_split:
            # 4개 GPU용 무작위 분할
            ds = ds.shuffled(seed=123)
            split_size = len(ds.keys) // 4
            start_idx = gpu * split_size
            end_idx = start_idx + split_size if gpu < 3 else len(ds.keys)
            ds = ds.change_keys(ds.keys[start_idx:end_idx])
        else:
            # 길이 기반 분할 (4개 GPU용)
            ds = ds.sorted_by_len(formatter=formatter, name='input', max_of_transposed=True)
            assignment = ([0,1,2,3]*ds.length())[:ds.length()][::-1]
            ds = ds.change_keys((np.array(ds.keys)[np.array(assignment)==gpu]).tolist())
    
    
    if train:
        # ===== 훈련용 데이터셋 준비 =====
        ds = ds.remove_replies()  # 답안 제거 (자가 지도 학습)
        # 데이터 증강
        ds = ds.augment(
            tp=True,  # 전치 변환
            rot=True,  # 회전 변환
            perm=perm_aug,  # 순열 증강
            n=(2 if arc_test_set.is_fake else train_epochs),  # 증강 횟수
            shfl_ex=True,  # 예제 셔플
            shfl_keys=True  # 키 셔플
        )
        # 최대 길이로 자르기 (새 토큰 생성 없음)
        ds = ds.cut_to_len(formatter=formatter, name='text', max_len=max_seq_length_train, max_new_tokens=0)
        # 가짜 테스트 세트인 경우 긴 것부터 정렬
        if arc_test_set.is_fake: 
            ds = ds.sorted_by_len(formatter=formatter, name='text', reverse=True)
    else:
        # 추론 시에도 4개 GPU로 분할
        ds = ds.sorted_by_len(formatter=formatter, name='input', max_of_transposed=True)
        ds = ds.split_multi_replies()
        
        # 4개 GPU로 균등 분할
        if gpu is not None:
            total_keys = len(ds.keys)
            split_size = total_keys // 4
            start_idx = gpu * split_size
            end_idx = start_idx + split_size if gpu < 3 else total_keys
            ds.keys = ds.keys[start_idx:end_idx]
        
        # 증강 및 인터리브
        ds = ds.augment(tp=True, rot=True, n=2, seed=42, perm=perm_aug, shfl_ex=True).interleave(len(ds.keys))
        ds = ds.cut_to_len(formatter=formatter, name='input', max_len=max_seq_length_infer)
        
        if arc_test_set.is_fake: 
            ds.keys = ds.keys[:32]  # 각 GPU당 32개씩
    
    return ds

def start_training(gpu):
    """
    지정된 GPU에서 훈련을 시작하는 함수
    
    Args:
        gpu: 사용할 GPU 번호
    """
    try:
        storage_path = f'{model_temp_storage}_gpu{gpu}'
        # GPU 0이거나 다중 GPU 모드이고, 저장 경로가 없는 경우에만 훈련
        if (gpu==0 or multi_gpu_train) and not os.path.exists(storage_path):
            with RemapCudaOOM():  # CUDA OOM 에러 처리
                # 모델과 포맷터 준비
                model, formatter = prepare_run(base_model, train=True, gpu=gpu)
                # 데이터셋 준비
                dataset = prepare_dataset(formatter, train=True, gpu=gpu if multi_gpu_train else None)
                
                # 훈련 실행
                model, trainer_stats = training_run(
                    model, formatter, dataset, 
                    store=storage_path,  # 모델 저장 경로
                    max_seq_length=max_seq_length_train,
                    grad_acc_fix=False,  # 그래디언트 누적 수정 비활성화
                    train_args=dict(
                        per_device_train_batch_size=2,  # 디바이스당 배치 크기
                        gradient_accumulation_steps=2,  # 그래디언트 누적 단계
                        warmup_steps=100,  # 워밍업 단계
                        num_train_epochs=1,  # 훈련 에포크 수
                        max_steps=20 if arc_test_set.is_fake else -1,  # 최대 스텝 (가짜 세트는 20스텝만)
                        learning_rate=1e-4,  # 학습률
                        embedding_learning_rate=1e-5,  # 임베딩 학습률
                        logging_steps=10,  # 로깅 간격
                        optim="adamw_8bit",  # 8비트 AdamW 옵티마이저
                        weight_decay=0.01,  # 가중치 감쇠
                        lr_scheduler_type='cosine',  # 코사인 학습률 스케줄러
                        seed=42,  # 랜덤 시드
                        output_dir=os.path.join(tmp_dir, 'checkpoints'),  # 체크포인트 저장 경로
                        save_strategy="no",  # 저장 전략 (저장 안함)
                        report_to='none',  # 리포팅 비활성화
                    ),
                )
                mem_info()  # 메모리 정보 출력
    finally: 
        # 훈련 완료 표시 파일 생성
        os.makedirs(f'{storage_path}_done', exist_ok=True)

def start_inference(gpu):
    """
    지정된 GPU에서 추론을 시작하는 함수
    
    Args:
        gpu: 사용할 GPU 번호
    """
    storage_path = f'{model_temp_storage}_gpu{gpu % 4 if multi_gpu_train else 0}'

    
    # 훈련 완료까지 대기
    while not os.path.exists(f'{storage_path}_done'): 
        time.sleep(15)
    
    with RemapCudaOOM():  # CUDA OOM 에러 처리
        # 훈련된 모델로 준비
        model, formatter = prepare_run(storage_path, gpu=gpu)
        # 추론용 데이터셋 준비
        dataset = prepare_dataset(formatter, train=False, gpu=gpu)
        
        # 단일 작업 프라이밍을 위한 재훈련기 설정
        retrainer = None if not prime_on_single_task else Retrainer(
            n=32,  # 훈련 샘플 수
            aug_opts=dict(perm=perm_aug, shfl_ex=True),  # 증강 옵션
            reload_state_dict=get_and_fix_peft_weights(storage_path),  # PEFT 가중치 재로드
            formatter=formatter,
            max_seq_length=max_seq_length_infer,
            grad_acc_fix=False,
            train_args=dict(
                per_device_train_batch_size=2,
                gradient_accumulation_steps=2,
                warmup_steps=4,
                num_train_epochs=1,
                learning_rate=1e-4,
                embedding_learning_rate=0,  # 임베딩 학습률 0 (고정)
                logging_steps=8,
                optim="adamw_8bit",
                weight_decay=0.00,  # 가중치 감쇠 없음
                lr_scheduler_type='constant',  # 상수 학습률
                seed=42,
                output_dir='tmp_output',
                save_strategy='no',
                report_to='none',
            ),
        )
        
        # 디코더 설정
        decoder = Decoder(
            formatter, 
            arc_test_set.split_multi_replies(), 
            n_guesses=2,  # 최대 추측 횟수
            prob_baseline=0.05  # 확률 기준선
        )
        
        # 추론 실행
        inference_run_v2(model, formatter, dataset, decoder, retrain=retrainer, **infer_params)
        
        # 증강 점수 계산 (필요한 경우)
        if use_aug_score or arc_test_set.is_fake: 
            decoder.calc_augmented_scores(model=model, store=score_temp_storage, **aug_score_params)
        
        mem_info()  # 메모리 정보 출력

class RemapCudaOOM:
    """CUDA Out of Memory 에러를 처리하는 컨텍스트 매니저"""
    
    def __enter__(self): 
        """컨텍스트 진입 시 아무것도 하지 않음"""
        pass
    
    def __exit__(self, exc_type, exc_value, traceback):
        """
        컨텍스트 종료 시 CUDA OOM 에러 처리
        
        CUDA 메모리 부족 에러가 발생하면 제출 파일을 생성하여
        채점 에러를 발생시킴 (Kaggle 환경에서의 안전장치)
        """
        oom_errors = [
            "CUDA out of memory", 
            "Make sure you have enough GPU RAM", 
            "does not fit any GPU's remaining memory"
        ]
        if exc_value and any(x in str(exc_value) for x in oom_errors):
            # 의도적으로 잘못된 제출 파일 생성
            with open('submission.json', 'w') as f: 
                f.write('cause submission scoring error')

In [ ]:
from common_stuff import *
import os
os.environ["WANDB_DISABLED"] = "true"

if not os.path.exists(os.path.join(tmp_dir, 'unsloth_installed')):  # unsloth offline install - https://stackoverflow.com/a/51646354
    !pip uninstall --yes torch accelerate
    !pip install --no-index --find-links=/kaggle/input/unsloth-2024-9-post4/wheelhouse unsloth
    #!pip uninstall --yes accelerate fastai torch torchaudio transformers
    #!pip install --no-index --find-links=/kaggle/input/unsloth-2024-10-7/wheelhouse unsloth  # do not use grad_acc_fix - trains very slow
    #!sed -i 's/if ((post_check - pre_check) >= 1).sum() > 1:/if False:/g' /opt/conda/lib/python3.10/site-packages/unsloth/models/llama.py
    # fix delay bug in get_statistics()
    !sed -i 's/^def get_statistics():/def get_statistics():\n if False:/g' /opt/conda/lib/python3.10/site-packages/unsloth/models/_utils.py
    # fix faulty unsloth multi-gpu detection
    !sed -i "s/raise RuntimeError('Unsloth currently does not support multi GPU setups - but we are working on it!')/pass/g" /opt/conda/lib/python3.10/site-packages/unsloth/tokenizer_utils.py /opt/conda/lib/python3.10/site-packages/unsloth/models/llama.py /opt/conda/lib/python3.10/site-packages/unsloth/models/vision.py
    os.makedirs(os.path.join(tmp_dir, 'unsloth_installed'), exist_ok=True)
    print('Unsloth installed & patched.')

for gpu in [0, 1, 2, 3]: 
    signal_path = f'{model_temp_storage}_gpu{gpu}_done'
    if os.path.exists(signal_path): os.rmdir(signal_path)

if arc_test_set.is_fake:  # cleanup? (for debugging)
    #!rm -R /kaggle/temp/finetuned_model*
    #!rm -R /kaggle/temp/inference_outputs
    #!rm -R /kaggle/temp/inference_scoring
    #!ls /kaggle/temp
    pass

In [9]:
%%python --bg --proc train_proc0
from common_stuff import *
start_training(gpu=0)

In [10]:
%%python --bg --proc train_proc1
from common_stuff import *
start_training(gpu=1)

In [11]:
%%python --bg --proc train_proc2
from common_stuff import *
start_training(gpu=2)

In [12]:
%%python --bg --proc train_proc3
from common_stuff import *
start_training(gpu=3)

In [13]:
%%python --bg --proc infer_proc0
from common_stuff import *
start_inference(gpu=0)

In [14]:
%%python --bg --proc infer_proc1
from common_stuff import *
start_inference(gpu=1)

In [15]:
%%python --bg --proc infer_proc2
from common_stuff import *
start_inference(gpu=2)

In [16]:
%%python --bg --proc infer_proc3
from common_stuff import *
start_inference(gpu=3)

In [ ]:
proc_exit_codes = await wait_for_subprocesses(
    train_proc0, train_proc1, train_proc2, train_proc3,
    infer_proc0, infer_proc1, infer_proc2, infer_proc3, 
    print_output=True or arc_test_set.is_fake)
print(f'*** Subprocesses exit codes: {proc_exit_codes}')
assert all(x==0 for x in proc_exit_codes)

In [ ]:
# write submission 부분 수정
from common_stuff import *
with RemapCudaOOM():
    model, formatter, dataset = None, MyFormatter(), None
    decoder = Decoder(formatter, arc_test_set.split_multi_replies(), n_guesses=2, frac_score=True)
    
    # 모든 GPU의 결과를 병합
    for gpu in range(4):
        gpu_store = f"{infer_temp_storage}_gpu{gpu}" if multi_gpu_train else infer_temp_storage
        if os.path.exists(gpu_store):
            decoder.from_store(gpu_store)
    
    if use_aug_score or arc_test_set.is_fake: 
        decoder.calc_augmented_scores(model=model, store=score_temp_storage, **aug_score_params)
    
    submission = arc_test_set.get_submission(decoder.run_selection_algo(submission_select_algo))
    with open('submission.json', 'w') as f: json.dump(submission, f)
    if arc_test_set.is_fake:
        decoder.benchmark_selection_algos(selection_algorithms)
        with open('submission.json') as f: reload_submission = json.load(f)
        print('*** Reload score:', arc_test_set.validate_submission(reload_submission))